In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
color = sns.color_palette() 
%matplotlib inline
sns.set(style='white', context='notebook', palette='deep')

# for seaborn issue:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore")
%precision 4

from datetime import datetime
from datetime import date

from dateutil.relativedelta import relativedelta

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [162]:
#DS_DF=pd.read_csv('/Users/sri.t/Desktop/Amex1/New/downstream_lavender_2 2.csv',sep=',')
DS_DF=pd.read_csv('/Users/sri.t/Desktop/Amex1/New/DS_across_files.csv',sep=',')

In [163]:
DS_DF_Mini=DS_DF[(DS_DF['LAV_ID1'].notnull())&(DS_DF['LAV_ID1'].isin(['LAV000000000431712','LAV000000000670214','LAV000000000181635', 'LAV000000000935734','LAV000000001172242']))][['LAV_ID1', 
        'Gender', 'IncomeCat','LOC','Income','Dob','PostCode','CHANNEL',
       'cancel_dt', 'cancel_type', 'detailed_fee_type', 'ACQ_TYPE', 'acq_chn',
       'prod_type', 'product_description','MR_Source_Code',
       'atvesupp', 'estab_dt', 'Bkt_Month', 'BtStatus', 'acct_naa', 'acct_aif',
       'acct_bill_active', 'acct_vol_can', 'acct_transf_can', 'acct_invol_can',
       'acct_col_can', 'total_attr', 'curr_bal_amt',
       'acct_bt_ind', 'bt_bal_amt', 'acct_spend_lcl','acct_supp_spend_lcl','revolving_ind', 'roc_amt',
       'roc_cnt', 'roc_supp_amt', 'roc_supp_cnt', 'acct_myca_active', 'optout',
       'myca_active_app', 'myca_app_ind', 'myca_active_mob',
       'APP_logon_iphone', 'APP_logon_ipad', 'APP_logon_android_m',
       'APP_logon_android_t', 'APP_logon_other', 'Tenure', 'AgeCat',
       'prod_tier', 'estab_mth','SuppStatus']]

In [164]:
#del DS_DF_Mini_1
del DS_DF

In [165]:
print (DS_DF_Mini.shape)
print (len(DS_DF_Mini['LAV_ID1'].unique()))

(185, 53)
5


In [166]:
#DS_DF_Mini.dtypes

In [167]:
# make all negative spends to zero along with associated counts
#DS_DF_Mini['acct_spend_lcl']=[0 if x<0 else x for x in DS_DF_Mini['acct_spend_lcl']]
#DS_DF_Mini['roc_cnt'][DS_DF_Mini['acct_spend_lcl'] == 0]=0
#DS_DF_Mini['acct_supp_spend_lcl']=[0 if x<0 else x for x in DS_DF_Mini['acct_supp_spend_lcl']]
#DS_DF_Mini['roc_supp_cnt'][DS_DF_Mini['acct_supp_spend_lcl'] == 0]=0
#DS_DF_Mini['curr_bal_amt']=[0 if x<0 else x for x in DS_DF_Mini['curr_bal_amt']]

In [168]:
for i in range(len(DS_DF_Mini.columns)):
    if (DS_DF_Mini.iloc[:,i].dtype == 'float64'):
        DS_DF_Mini.iloc[:,i]=DS_DF_Mini.iloc[:,i].fillna(value=0)           

DS_DF_Mini['atvesupp']=DS_DF_Mini['atvesupp'].fillna(value=0)

In [169]:
DS_DF_Mini['BtStatus_1']=[1 if x=='BT Active' else 0 for x in DS_DF_Mini['BtStatus']]

In [170]:
DS_DF_Mini['SuppStatus_1']=[1 if x=='Yes Supp' else 0 for x in DS_DF_Mini['SuppStatus']]

In [171]:
DS_DF_Mini['count']=DS_DF_Mini.groupby('LAV_ID1').cumcount() + 1

In [172]:
DS_DF_Mini['max_count']=DS_DF_Mini.groupby('LAV_ID1')['count'].transform(max)

In [173]:
DS_DF_Mini['ticket_size']=DS_DF_Mini['acct_spend_lcl']/DS_DF_Mini['roc_cnt']

In [174]:
DS_DF_Mini['ticket_size']=[0 if x==np.inf else x for x in DS_DF_Mini['ticket_size']]

In [175]:
df1=DS_DF_Mini.groupby(['LAV_ID1']).agg({'acct_spend_lcl':'mean',
    'roc_cnt':'mean','acct_supp_spend_lcl':'mean','roc_supp_cnt':'mean','ticket_size':'mean',
    'myca_active_app':'mean','optout':'sum','acct_myca_active':'sum','myca_app_ind':'sum',
    'SuppStatus_1':'sum','revolving_ind':'sum','BtStatus_1':'sum'}).reset_index()

In [176]:
df2=DS_DF_Mini[DS_DF_Mini['count']==DS_DF_Mini['max_count']].astype(str).groupby(['LAV_ID1', 'AgeCat','Gender','IncomeCat','estab_dt',
    'cancel_dt','PostCode', 'detailed_fee_type','ACQ_TYPE','acq_chn','product_description','prod_tier']).size().reset_index()

In [177]:
df2.drop(columns=[0],axis=1,inplace=True)

In [178]:
df2.shape

(5, 12)

In [179]:
df1.shape

(5, 13)

In [180]:
df=df1.merge(df2,on='LAV_ID1',how='inner')

In [181]:
df.shape

(5, 24)

In [182]:
df.head()

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,ticket_size,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier
0,LAV000000000181635,1175.513514,13.270270,0.0,0.0,91.172212,0.00000,0.0,3.0,0.0,0,0.0,0,45 - 55,M,$65k - $85k,09NOV2001,nan,2122.0,Free For Life / No Fee,nan,nan,AFFINITY GOLD PREMIUM,nan
1,LAV000000000431712,0.108108,0.054054,0.0,0.0,2.000000,0.00000,0.0,0.0,0.0,0,0.0,0,45 - 55,F,$0 - $65k,16DEC2003,JAN2014,3765.0,Free For Life / No Fee,nan,nan,AFFINITY GOLD PREMIUM,nan
2,LAV000000000670214,763.486486,14.837838,0.0,0.0,53.535266,11.27027,37.0,32.0,36.0,0,2.0,0,25 - 35,U,$65k - $85k,02APR2012,nan,6060.0,Full Fee,External,Interactive,PLATINUM EDGE,Mid
3,LAV000000000935734,572.297297,4.297297,0.0,0.0,155.149485,0.00000,0.0,26.0,0.0,37,30.0,0,55 - 65,F,$0 - $65k,25FEB2015,nan,6108.0,Full Fee,Internal,Direct Mail,DAVID JONES PLATINUM,Premium
4,LAV000000001172242,0.000000,0.000000,0.0,0.0,NaN,0.00000,0.0,0.0,0.0,0,0.0,0,55 - 65,F,$0 - $65k,26MAY2014,OCT2014,3145.0,Full Fee,External,F2F,DAVID JONES GPC,Mid


In [183]:
#Final shape
#DS 1 - 61696, 23 -- 181,634 , 23
#DS 2 - 99585, 23 -- 250,076 , 23
#DS 3 - 103323, 23 -- 238,501 , 23
#DS 4 - 95939, 23 --  265,519 , 23
#DS 5 - 112109, 23 --  236 507 , 23
#DS 6 - 33606, 23 --  133,342 , 23
#DS 7 - 2,23 -- 5,23

In [184]:
df.to_csv('/Users/sri.t/Desktop/AMEX1/For Tony/DS_7_1.csv',header=True,sep=',',encoding='utf-8')

In [33]:
### Combine all DS files

In [185]:
f1=pd.read_csv('/Users/sri.t/Desktop/AMEX1/For Tony/DS_1_1.csv',sep=',')
f2=pd.read_csv('/Users/sri.t/Desktop/AMEX1/For Tony/DS_2_1.csv',sep=',')
f3=pd.read_csv('/Users/sri.t/Desktop/AMEX1/For Tony/DS_3_1.csv',sep=',')
f4=pd.read_csv('/Users/sri.t/Desktop/AMEX1/For Tony/DS_4_1.csv',sep=',')
f5=pd.read_csv('/Users/sri.t/Desktop/AMEX1/For Tony/DS_5_1.csv',sep=',')
f6=pd.read_csv('/Users/sri.t/Desktop/AMEX1/For Tony/DS_6_1.csv',sep=',')
f7=pd.read_csv('/Users/sri.t/Desktop/AMEX1/For Tony/DS_7_1.csv',sep=',')

In [186]:
print (f1.shape)
print (f2.shape)
print (f3.shape)
print (f4.shape)
print (f5.shape)
print (f6.shape)
print (f7.shape)

(181634, 25)
(250076, 25)
(238501, 25)
(265519, 25)
(236507, 25)
(133342, 25)
(5, 25)


In [187]:
df=pd.concat([f1,f2,f3,f4,f5,f6,f7])

In [188]:
df.shape

(1305584, 25)

In [189]:
df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [190]:
df.min()

LAV_ID1                LAV000000000000001
acct_spend_lcl                   -933.135
roc_cnt                                 0
acct_supp_spend_lcl              -567.568
roc_supp_cnt                            0
ticket_size                          -inf
myca_active_app                         0
optout                                  0
acct_myca_active                        0
myca_app_ind                            0
SuppStatus_1                            0
revolving_ind                           0
BtStatus_1                              0
AgeCat                            18 - 25
Gender                                  F
estab_dt                        01APR1959
detailed_fee_type         First Year Free
product_description         AFFINITY BLUE
dtype: object

In [191]:
#df.isnull().sum()

In [192]:
df['acct_spend_lcl']=[0 if x<0 else x for x in df['acct_spend_lcl']]

In [193]:
df['acct_supp_spend_lcl']=[0 if x<0 else x for x in df['acct_supp_spend_lcl']]

In [201]:
df.head()

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,ticket_size,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned
0,LAV000000000000001,0.000000,0.000000,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,01NOV1999,SEP2014,NaN,Full Fee,NaN,NaN,GREEN,NaN,1
1,LAV000000000000002,119.648649,1.324324,0.0,0.0,72.058333,0.0,37.0,1.0,0.0,0,0.0,0,Unkn,U,NaN,01NOV1999,NaN,NaN,Free For Life / No Fee,NaN,NaN,GREEN,NaN,0
2,LAV000000000000003,70.945946,0.675676,0.0,0.0,77.487179,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,01NOV1999,NOV2015,NaN,Full Fee,NaN,NaN,GREEN,NaN,1
3,LAV000000000000004,4073.702703,57.378378,0.0,0.0,79.123905,0.0,0.0,36.0,0.0,0,0.0,0,Unkn,U,NaN,01NOV1999,NaN,NaN,Full Fee,NaN,NaN,GREEN,NaN,0
4,LAV000000000000005,3212.135135,10.729730,0.0,0.0,307.737615,0.0,36.0,18.0,0.0,0,0.0,0,Unkn,U,NaN,01NOV1989,NaN,NaN,Full Fee,NaN,NaN,GREEN,NaN,0


In [199]:
df['Churned']=[0 if x=='nan' else 1 for x in df['cancel_dt'].apply(lambda x: str(x))]

In [202]:
df['estab_dt']=[datetime.strptime(x, '%d%b%Y') for x in df['estab_dt']]

In [204]:
#df['days_active']=[i.days for i in (datetime(year=2017,month=11,day=1)-df['estab_dt'])]

In [205]:
df['cancel_dt']=df['cancel_dt'].fillna(value="JAN2100")

In [206]:
df['cancel_dt']=['01'+x for x in df['cancel_dt']]

In [207]:
df['cancel_dt']=[datetime.strptime(x, '%d%b%Y') for x in df['cancel_dt']]

In [208]:
df['estab_dt_month']=[x.month for x in df['estab_dt']]
df['estab_dt_year']=[x.year for x in df['estab_dt']]

In [209]:
def f3(x):
    return datetime(year=x['estab_dt_year'],month=x['estab_dt_month'],day=1)

In [210]:
df['estab_dt_1']=df[['estab_dt_year','estab_dt_month']].apply(f3,axis=1)

In [215]:
def f1(x):
    if (x['cancel_dt']==datetime(year=2100,month=1,day=1)):
        g=(relativedelta(datetime(year=2017,month=11,day=1),x['estab_dt_1']).months)+(relativedelta(datetime(year=2017,month=11,day=1),x['estab_dt_1']).years*12)
    else:
        g=(relativedelta(x['cancel_dt'],x['estab_dt_1']).months)+(relativedelta(x['cancel_dt'],x['estab_dt_1']).years*12)
    return g       

In [216]:
df['months_active']=df[['cancel_dt','estab_dt_1']].apply(f1,axis=1)

In [217]:
df['tenure_bin']=['1. 0-1' if x<12 else '2. 1-2' if 12<=x<24 else '3. 2-3' if 24<=x<36 else '4. 3-4' if 36<=x<48 else '5. 4-5' if 48<=x<60 else '6. 5-10' if 60<=x<120 else '7. 10-15' if 120<=x<180 else '8.15-20' if 180<=x<240 else '8. >20' for x in df['months_active'] ]

In [218]:
df.head()

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,ticket_size,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin
0,LAV000000000000001,0.000000,0.000000,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15
1,LAV000000000000002,119.648649,1.324324,0.0,0.0,72.058333,0.0,37.0,1.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Free For Life / No Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20
2,LAV000000000000003,70.945946,0.675676,0.0,0.0,77.487179,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2015-11-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,192,8.15-20
3,LAV000000000000004,4073.702703,57.378378,0.0,0.0,79.123905,0.0,0.0,36.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20
4,LAV000000000000005,3212.135135,10.729730,0.0,0.0,307.737615,0.0,36.0,18.0,0.0,0,0.0,0,Unkn,U,NaN,1989-11-01,2100-01-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,0,11,1989,1989-11-01,336,8. >20


In [219]:
df['acct_spend_zero_flag']=[1 if x==0 else 0 for x in df['acct_spend_lcl']]

In [220]:
df['acct_spend_quartile']=0
df['acct_spend_decile']=0
df['acct_spend_pentile']=0

In [221]:
df['acct_spend_pentile'][df['acct_spend_zero_flag']==0]=pd.qcut(df[df['acct_spend_zero_flag']==0]['acct_spend_lcl'], 5, labels=False) + 1
df['acct_spend_quartile'][df['acct_spend_zero_flag']==0]=pd.qcut(df[df['acct_spend_zero_flag']==0]['acct_spend_lcl'], 4, labels=False) + 1
df['acct_spend_decile'][df['acct_spend_zero_flag']==0]=pd.qcut(df[df['acct_spend_zero_flag']==0]['acct_spend_lcl'], 10, labels=False) + 1

In [222]:
df['acct_spend_quartile'][df['acct_spend_zero_flag']==1]=-1
df['acct_spend_decile'][df['acct_spend_zero_flag']==1]=-1
df['acct_spend_pentile'][df['acct_spend_zero_flag']==1]=-1

In [223]:
#rr=df['cancel_dt'].astype(str).value_counts()

In [224]:
#rr.head()

In [225]:
#df.head()

In [226]:
df=df.set_index('LAV_ID1')

In [227]:
#df.to_csv('/Users/sri.t/Desktop/AMEX1/For Tony/DS_all.csv',header=True,sep=',',encoding='utf-8')

In [228]:
#df.isnull().sum()

### Email

In [229]:
df2=pd.read_csv('/Users/sri.t/Desktop/Amex1/New/email_data_lavender_v2.csv',sep=',')

In [230]:
df2.head()

,month_year,total_sent,total_opens,total_clicks,email_open_rate,email_ctr,LAV_ID1,amex_cust_id,STAR_ID
0,201512,1,1,0,1.0,0.0,LAV000000000000002,3176319073,3.699308e+10
1,201708,1,1,0,1.0,0.0,LAV000000000000002,3176319073,3.699308e+10
2,201704,1,2,0,1.0,0.0,LAV000000000000002,3176319073,3.699308e+10
3,201702,1,1,0,1.0,0.0,LAV000000000000002,3176319073,3.699308e+10
4,201511,2,4,0,1.0,0.0,LAV000000000000002,3176319073,3.699308e+10


In [231]:
df2_1=df2.groupby('LAV_ID1').agg({'total_sent':'mean','total_opens':'mean','total_clicks':'mean'})

In [232]:
df2_1.shape

(884580, 3)

In [233]:
df2_1.head()

,total_sent,total_opens,total_clicks
LAV_ID1,,,
LAV000000000000002,1.125000,1.625000,0.000000
LAV000000000000003,2.636364,0.454545,0.090909
LAV000000000000004,4.416667,1.638889,0.027778
LAV000000000000005,1.125000,1.000000,0.250000
LAV000000000000006,1.125000,1.125000,0.000000


### MR

In [234]:
df3=pd.read_csv('/Users/sri.t/Desktop/Amex1/New/MRrewardsdatav2.csv',sep=',')

In [235]:
df3=df3[df3['placed_date']>=201412].reset_index()

In [236]:
print (df3.shape)
print (len(df3['LAV_ID1'].unique()))
print (df3['placed_date'].min())
print (df3['placed_date'].max())

(1769272, 14)
356149
201412
201711


In [237]:
#let all negative MR redemptions to zero
df3['rdm_pnt']=[0 if x<0 else x for x in df3['rdm_pnt']]
df3['rdm_cnt'][df3['rdm_pnt'] == 0]=0

In [238]:
df3=df3[df3['LAV_ID1'].notnull()]
print (df3.shape)

(1651915, 14)


In [239]:
df3.head()

,index,LAV_ID1,amex_cust_id,MR_LINK_TYPE_CD,MR_MBR_ID,MR_TIER_CD,TIER_SHORT_NM,MR_Benefit,Level1,Level2,Level3,placed_date,rdm_cnt,rdm_pnt
0,0,LAV000000000000003,1.83374e+09,P,36993270124,B06,Ascent,Shop with Points (DJ),Non Air,PWP,David Jones Pay with Points Online,201507,2,30712
1,2,LAV000000000000004,1.83373e+09,P,36993230105,B06,Ascent,Air Transfer,Air,VFF,Velocity- Virgin Australia,201412,1,2642
2,3,LAV000000000000005,3.1763e+09,P,36993280532,B06,Ascent,Air Transfer,Air,Air - Foreign,Singapore Airlines,201607,1,200000
3,7,LAV000000000000016,7.87547e+09,P,36962680081,B06,Ascent,Air Transfer,Air,VFF,Velocity- Virgin Australia,201502,1,26783
4,8,LAV000000000000016,7.87547e+09,P,36962680081,B06,Ascent,Air Transfer,Air,VFF,Velocity- Virgin Australia,201506,1,9108


In [268]:
df3_agg=df3.groupby(['LAV_ID1','placed_date','MR_Benefit','Level1','Level2','Level3']).agg({'rdm_cnt':['sum'],'rdm_pnt':['sum']}, as_index=False).reset_index()
ind = pd.Index([e[0] + e[1] for e in df3_agg.columns.tolist()])
df3_agg.columns=ind

In [300]:
#df3_agg.shape

In [301]:
#len(df3['LAV_ID1'].unique())  #356148

In [270]:
y=df3_agg['Level2'].unique()

In [279]:
f=df3_agg.groupby('LAV_ID1').agg({'rdm_cntsum':'mean','rdm_pntsum':'mean'})

In [286]:
for j in y:
    f11=df3_agg[df3_agg['Level2']==j].groupby('LAV_ID1').agg({'rdm_pntsum':'mean'})
    f11=f11.fillna(value=0)
    #ind = pd.Index([e[0] + '_' + e[1] for e in f11.columns.tolist()])
    ind1=[j+'_'+x for x in f11.columns]
    ind = [w.replace('rdm_pntsum', 'pnt') for w in ind1]
    f11.columns= ind
    #f11.columns=[j+"_pnt_Q1",j+"_pnt_Q2",j+"_pnt_Q3",j+"_pnt_Q4"]
    f=f.merge(f11,how='outer',left_index=True,right_index=True)

In [289]:
for j in y:
    f11=df3_agg[df3_agg['Level2']==j].groupby('LAV_ID1').agg({'rdm_cntsum':'mean'})
    f11=f11.fillna(value=0)
    #ind = pd.Index([e[0] + '_' + e[1] for e in f11.columns.tolist()])
    ind1=[j+'_'+x for x in f11.columns]
    ind = [w.replace('rdm_cntsum', 'cnt') for w in ind1]
    f11.columns= ind
    #f11.columns=[j+"_pnt_Q1",j+"_pnt_Q2",j+"_pnt_Q3",j+"_pnt_Q4"]
    f=f.merge(f11,how='outer',left_index=True,right_index=True)

In [290]:
f.tail()

,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt
LAV_ID1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
LAV000000001305522,1.0,1350.0,NaN,NaN,NaN,NaN,NaN,NaN,1350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAV000000001305523,1.0,1207.0,NaN,NaN,NaN,NaN,NaN,1064.0,1350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAV000000001305524,1.0,1350.0,NaN,NaN,NaN,NaN,NaN,NaN,1350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAV000000001305576,1.0,1175.0,NaN,NaN,NaN,NaN,NaN,1000.0,1350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAV000000001305577,1.0,1350.0,NaN,NaN,NaN,NaN,NaN,NaN,1350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [299]:
f.shape

(356148, 40)

In [302]:
del df3
del df3_agg

### PIB

In [244]:
df4=pd.read_csv('/Users/sri.t/Desktop/Amex1/New/PIB_PCS_lavender_v2.csv',sep=',')

In [245]:
df4=df4[df4['LAV_ID1'].notnull()]
print (df4.shape)
print (len(df4['LAV_ID1'].unique()))

(26620546, 7)
865532


In [246]:
df4['PIB_MONTH_END']=[datetime.strptime(x, '%d%b%Y') for x in df4['PIB_MONTH_END']]

In [247]:
df4_1=df4[(df4['PIB_MONTH_END']>=datetime(year=2014,month=12,day=31))&(df4['PIB_MONTH_END']<=datetime(year=2017,month=11,day=30))]

In [248]:
df4_1['Latest_PIB']=[0 if x <0 else x for x in df4_1['Latest_PIB']]

In [249]:
del df4

In [250]:
df4_1.shape

(19831540, 7)

In [251]:
df4_2=df4_1.groupby('LAV_ID1').agg({'Latest_PIB':'mean'})

In [252]:
df4_2.shape

(774290, 1)

In [253]:
df4_2.head()

,Latest_PIB
LAV_ID1,
LAV000000000000001,33468.500000
LAV000000000000003,14422.266667
LAV000000000000004,65627.888889
LAV000000000000005,124840.583333
LAV000000000000007,2509.000000


In [254]:
df4_2.min()

Latest_PIB    0.0
dtype: float64

### Combine DS, Email, MR, PIB

In [303]:
final_df=df.merge(df2_1,how='left',left_index=True,right_index=True).merge(f,how='left',left_index=True,right_index=True).merge(df4_2,how='left',left_index=True,right_index=True).reset_index()

In [304]:
final_df.shape

(1305584, 78)

In [305]:
final_df.head()

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,ticket_size,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB
0,LAV000000000000001,0.000000,0.000000,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33468.500000
1,LAV000000000000002,119.648649,1.324324,0.0,0.0,72.058333,0.0,37.0,1.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Free For Life / No Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20,0,2,4,2,1.125000,1.625000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LAV000000000000003,70.945946,0.675676,0.0,0.0,77.487179,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2015-11-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,192,8.15-20,0,1,3,2,2.636364,0.454545,0.090909,2.0,30712.0,30712.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14422.266667
3,LAV000000000000004,4073.702703,57.378378,0.0,0.0,79.123905,0.0,0.0,36.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20,0,4,10,5,4.416667,1.638889,0.027778,1.0,2642.0,NaN,2642.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65627.888889
4,LAV000000000000005,3212.135135,10.729730,0.0,0.0,307.737615,0.0,36.0,18.0,0.0,0,0.0,0,Unkn,U,NaN,1989-11-01,2100-01-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,0,11,1989,1989-11-01,336,8. >20,0,4,10,5,1.125000,1.000000,0.250000,1.0,200000.0,NaN,NaN,200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124840.583333


In [307]:
final_df['MR_Yes']=[0 if (np.isnan(x)) else 1 for x in final_df['rdm_cntsum']]

In [308]:
final_df['EM_Yes']=[0 if (np.isnan(x)) else 1 for x in final_df['total_sent']]

In [309]:
final_df['PIB_Yes']=[0 if (np.isnan(x)) else 1 for x in final_df['Latest_PIB']]

In [310]:
final_df['E_O/S']=final_df['total_opens']/final_df['total_sent']
final_df['E_C/O']=final_df['total_clicks']/final_df['total_opens']

In [311]:
final_df['MR_pnt_pentile']=0
final_df['MR_pnt_decile']=0
final_df['MR_pnt_quartile']=0

In [313]:
final_df['MR_pnt_pentile'][final_df['MR_Yes']==1]=pd.qcut(final_df[final_df['MR_Yes']==1]['rdm_pntsum'], 5, labels=False) + 1
final_df['MR_pnt_decile'][final_df['MR_Yes']==1]=pd.qcut(final_df[final_df['MR_Yes']==1]['rdm_pntsum'], 10, labels=False) + 1
final_df['MR_pnt_quartile'][final_df['MR_Yes']==1]=pd.qcut(final_df[final_df['MR_Yes']==1]['rdm_pntsum'], 4, labels=False) + 1

In [314]:
final_df['MR_pnt_pentile'][final_df['MR_Yes']==0]=-1
final_df['MR_pnt_decile'][final_df['MR_Yes']==0]=-1
final_df['MR_pnt_quartile'][final_df['MR_Yes']==0]=-1

In [315]:
final_df['E_O/S_pentile']=0
final_df['E_O/S_decile']=0
final_df['E_O/S_quartile']=0
final_df['E_C/O_pentile']=0
final_df['E_C/O_decile']=0
final_df['E_C/O_quartile']=0

In [316]:
final_df['E_O/S_pentile'][final_df['EM_Yes']==1]=pd.qcut(final_df[final_df['EM_Yes']==1]['E_O/S'], 5, labels=False,duplicates='drop') + 1
final_df['E_O/S_decile'][final_df['EM_Yes']==1]=pd.qcut(final_df[final_df['EM_Yes']==1]['E_O/S'], 10, labels=False,duplicates='drop') + 1
final_df['E_O/S_quartile'][final_df['EM_Yes']==1]=pd.qcut(final_df[final_df['EM_Yes']==1]['E_O/S'], 4, labels=False,duplicates='drop') + 1
final_df['E_C/O_pentile'][final_df['EM_Yes']==1]=pd.qcut(final_df[final_df['EM_Yes']==1]['E_C/O'], 5, labels=False,duplicates='drop') + 1
final_df['E_C/O_decile'][final_df['EM_Yes']==1]=pd.qcut(final_df[final_df['EM_Yes']==1]['E_C/O'], 10, labels=False,duplicates='drop') + 1
final_df['E_C/O_quartile'][final_df['EM_Yes']==1]=pd.qcut(final_df[final_df['EM_Yes']==1]['E_C/O'], 4, labels=False,duplicates='drop') + 1

In [317]:
final_df['E_O/S_pentile'][final_df['EM_Yes']==0]=-1
final_df['E_O/S_decile'][final_df['EM_Yes']==0]=-1
final_df['E_O/S_quartile'][final_df['EM_Yes']==0]=-1
final_df['E_C/O_pentile'][final_df['EM_Yes']==0]=-1
final_df['E_C/O_decile'][final_df['EM_Yes']==0]=-1
final_df['E_C/O_quartile'][final_df['EM_Yes']==0]=-1

In [318]:
final_df['PIB_pentile']=0
final_df['PIB_decile']=0
final_df['PIB_quartile']=0

In [319]:
final_df['PIB_pentile'][final_df['PIB_Yes']==0]=-1
final_df['PIB_decile'][final_df['PIB_Yes']==0]=-1
final_df['PIB_quartile'][final_df['PIB_Yes']==0]=-1

In [321]:
final_df['PIB_pentile'][final_df['PIB_Yes']==1]=pd.qcut(final_df[final_df['PIB_Yes']==1]['Latest_PIB'], 5, labels=False,duplicates='drop') + 1
final_df['PIB_decile'][final_df['PIB_Yes']==1]=pd.qcut(final_df[final_df['PIB_Yes']==1]['Latest_PIB'], 10, labels=False,duplicates='drop') + 1
final_df['PIB_quartile'][final_df['PIB_Yes']==1]=pd.qcut(final_df[final_df['PIB_Yes']==1]['Latest_PIB'], 4, labels=False,duplicates='drop') + 1

In [52]:
def func(x):
    return x[0:4]

In [55]:
def func1(x):
    return x[5:7]

In [324]:
g=final_df[final_df['Churned']==1]['cancel_dt'].apply(lambda x: str(x)).apply(func1)

In [325]:
final_df['Churned_Month']=np.nan
final_df['Churned_Year']=np.nan

In [326]:
final_df['Churned_Month'][final_df['Churned']==1]=g

In [327]:
h=final_df[final_df['Churned']==1]['cancel_dt'].apply(lambda x: str(x)).apply(func)

In [328]:
final_df['Churned_Year'][final_df['Churned']==1]=h

In [329]:
final_df['Churned'].value_counts()

0    799324
1    506260
Name: Churned, dtype: int64

In [330]:
final_df.shape

(1305584, 97)

In [10]:
is_df=pd.read_csv('/Users/sri.t/Desktop/AMEX1/For Tony/Industry_Spend_prepared.csv',sep=',')

In [11]:
is_df.shape

(1305584, 20)

In [21]:
#is_df.isnull().sum()

In [14]:
is_df.head()

,LAV_ID1,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio
0,LAV000000000000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,LAV000000000000002,140.417647,0.000000,0.000000,0.000000,0.000000,0.000000,1.411765,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,LAV000000000000003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,LAV000000000000004,2431.156471,1704.258824,1360.968235,37.050000,129.756471,0.000000,35.117647,3.647059,24.823529,0.941176,1.941176,0.000000,0.429291,0.300936,0.240318,0.006542,0.022912,0.000000
4,LAV000000000000005,1210.732353,226.405294,797.981765,55.823529,83.991176,6.758824,6.235294,2.352941,7.764706,0.588235,0.294118,0.176471,0.508349,0.095061,0.335048,0.023439,0.035265,0.002838


In [13]:
is_df.drop(columns=['Unnamed: 0'],inplace=True,axis=1)

In [30]:
is_df['IS_Yes']=1

In [128]:
final_df1=df.merge(is_df,on='LAV_ID1',how='inner')

In [129]:
final_df1.shape

(1305584, 117)

In [130]:
final_df1.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [131]:
final_df1.shape

(1305584, 116)

In [132]:
g=final_df1[final_df1['Churned']==1]['cancel_dt'].apply(lambda x: str(x)).apply(func1)

In [133]:
final_df1['Churned_Month'][final_df1['Churned']==1]=g

In [134]:
h=final_df1[final_df1['Churned']==1]['cancel_dt'].apply(lambda x: str(x)).apply(func)

In [135]:
final_df1['Churned_Year'][final_df1['Churned']==1]=h

In [111]:
#final_df1['ticket_size']=[0 if x<0 else x for x in final_df1['ticket_size']]

In [107]:
#final_df1['ticket_size']=[0 if x=='nan' else x for x in final_df1['ticket_size'].apply(lambda x: str(x))]

In [158]:
#final_df1['ticket_size']=final_df1['ticket_size'].astype(float)

In [157]:
#final_df1.isnull().sum()

In [156]:
#final_df1.dtypes

In [160]:
final_df1.head()

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes
0,LAV000000000000001,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33468.500000,0,0,1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,4,7,3,09,2014,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1
1,LAV000000000000002,119.648649,1.324324,0.0,0.0,0.0,37.0,1.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Free For Life / No Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20,0,2,4,2,1.125000,1.625000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1.444444,0.000000,-1,-1,-1,5,9,4,1.0,1.0,1.0,-1,-1,-1,NaN,NaN,140.417647,0.000000,0.000000,0.000000,0.000000,0.000000,1.411765,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
2,LAV000000000000003,70.945946,0.675676,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2015-11-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,192,8.15-20,0,1,3,2,2.636364,0.454545,0.090909,2.0,30712.0,30712.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14422.266667,1,1,1,0.172414,0.200000,4,7,3,2,2,1,3.0,4.0,2.0,3,5,3,11,2015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1
3,LAV000000000000004,4073.702703,57.378378,0.0,0.0,0.0,0.0,36.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20,0,4,10,5,4.416667,1.638889,0.027778,1.0,2642.0,NaN,2642.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65627.888889,1,1,1,0.371069,0.016949,1,1,1,2,3,2,1.0,1.0,1.0,5,8,4,NaN,NaN,2431.156471,1704.258824,1360.968235,37.050000,129.756471,0.000000,35.117647,3.647059,24.823529,0.941176,1.941176,0.000000,0.429291,0.300936,0.240318,0.006542,0.022912,0.000000,1
4,LAV000000000000005,3212.135135,10.729730,0.0,0.0,0.0,36.0,18.0,0.0,0,0.0,0,Un

In [159]:
final_df1.drop(columns=['ticket_size'],axis=1,inplace=True)

In [161]:
final_df1.shape

(1305584, 115)

In [153]:
#final_df1[(final_df1['ticket_size']==0)&(final_df1['acct_spend_decile']!=-1)].shape

In [154]:
#final_df1[(final_df1['ticket_size']==0)&(final_df1['acct_spend_decile']!=-1)].tail()

In [149]:
#final_df1[(final_df1['ticket_size']==-np.inf)&(final_df1['acct_spend_decile']!=-1)].tail()

In [148]:
#final_df1[(final_df1['ticket_size']==-np.inf)&(final_df1['acct_spend_decile']!=-1)].shape

In [136]:
#x=final_df1['ticket_size'].value_counts()

In [150]:
#x.head()

In [151]:
#final_df1[final_df1['ticket_size'].isnull()].shape

In [152]:
#final_df1[final_df1['ticket_size'].notnull()].shape

In [338]:
#cols=['LAV_ID1','Gender','AgeCat','IncomeCat','detailed_fee_type','product_description','prod_tier','PostCode','ACQ_TYPE','acq_chn',
#      'cancel_dt','Churned','Churned_Month','Churned_Year',
#      'estab_dt','estab_dt_month','estab_dt_year',
#      'optout','acct_myca_active','myca_app_ind','BtStatus_1','revolving_ind','SuppStatus_1',
#      'acct_spend_lcl','acct_spend_zero_flag','acct_spend_quartile','acct_spend_pentile','acct_spend_decile',
#      'acct_supp_spend_lcl',
#      'roc_cnt',
#      'roc_supp_cnt',
#      'myca_active_app',
#      'MR_Yes','rdm_cnt', 'rdm_pnt','MR_pnt_pentile', 'MR_pnt_decile', 'MR_pnt_quartile',
#      'EM_Yes','total_sent', 'total_opens','total_clicks','E_O/S', 'E_C/O',
#      'E_O/S_pentile', 'E_O/S_decile', 'E_O/S_quartile',
#      'E_C/O_pentile', 'E_C/O_decile', 'E_C/O_quartile',
#      'PIB_Yes','Latest_PIB','PIB_pentile','PIB_decile', 'PIB_quartile']

In [23]:
#final_df.to_csv('/Users/sri.t/Desktop/AMEX1/For Tony/amex_churners_estimates_3.csv',header=True,sep=',',encoding='utf-8')

In [162]:
final_df1.to_csv('/Users/sri.t/Desktop/AMEX1/For Tony/amex_churners_estimates_4.csv',header=True,sep=',',encoding='utf-8')

In [163]:
print (final_df1['MR_Yes'].value_counts())
print (final_df1['EM_Yes'].value_counts())
print (final_df1['PIB_Yes'].value_counts())
print (final_df1['IS_Yes'].value_counts())
print (final_df1['Churned'].value_counts())
print (final_df1['acct_spend_decile'].value_counts())

0    949436
1    356148
Name: MR_Yes, dtype: int64
1    884580
0    421004
Name: EM_Yes, dtype: int64
1    774290
0    531294
Name: PIB_Yes, dtype: int64
1    1305584
Name: IS_Yes, dtype: int64
0    799324
1    506260
Name: Churned, dtype: int64
-1     403578
 1      90264
 4      90213
 10     90201
 8      90201
 9      90199
 6      90199
 7      90196
 5      90191
 2      90182
 3      90160
Name: acct_spend_decile, dtype: int64


In [3]:
DS_DF=pd.read_csv('/Users/sri.t/Desktop/amex_churners_estimates_4.csv',sep=',')

In [4]:
DS_DF.shape

(1305584, 232)

In [5]:
DS_DF.head()

,Unnamed: 0,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag,all_MR_flag,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,B2B_flag,all_flag,acct_roc_decile,Churned_month,Churned_year,churne_month_year,acct_spend_decile_1,Adv_Publ,Air_Dom,Air_Intl,Air_Net,ATO,Car_Rental,Computer_Software,Consumables,Costco,Courier_Freight,Cruise,Dining,DJ_Prop_BB,DJ_GPC_BB,Education,Entertainment,Everything_Else,Financial_Services,Gas,Healthcare,Insurance,Lodging,Other_Air,Other_Gov,Other_Travel,Outbound,Personal_Services,QF_Air,Retail_DepStores,SBS_B2B,Supermarkets,Coles,Woolworths,Taxi_Limo,Telecomms,Tolls_Private,Transportation,Travel_Agents,Utilities,VA_Air,total,Adv_Publ_ratio,Air_Dom_ratio,Air_Intl_ratio,Air_Net_ratio,ATO_ratio,Car_Rental_ratio,Computer_Software_ratio,Consumables_ratio,Costco_ratio,Courier_Freight_ratio,Cruise_ratio,Dining_ratio,DJ_Prop_BB_ratio,DJ_GPC_BB_ratio,Education_ratio,Entertainment_ratio,Everything_Else_ratio,Financial_Services_ratio,Gas_ratio,Healthcare_ratio,Insurance_ratio,Lodging_ratio,Other_Air_ratio,Other_Gov_ratio,Other_Travel_ratio,Outbound_ratio,Personal_Services_ratio,QF_Air_ratio,Retail_DepStores_ratio,SBS_B2B_ratio,Supermarkets_ratio,Coles_ratio,Woolworths_ratio,Taxi_Limo_ratio,Telecomms_ratio,Tolls_Private_ratio,Transportation_ratio,Travel_Agents_ratio,Utilities_ratio,VA_Air_ratio,count,supp_status,cluster_label,all_zero_3year
0,0,LAV000000000000001,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33468.500000,0,0,1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,4,7,3,9.0,2014.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,9,2014,201409,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,998.0,NaN
1,1,LA

In [5]:
#DS_DF.isnull().sum()

In [6]:
DS_DF['PWP_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['PWP_pnt']]
DS_DF['VFF_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['VFF_pnt']]
DS_DF['AirForeign_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Air - Foreign_pnt']]

In [7]:
DS_DF['GiftCard_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Gift Card_pnt']]
DS_DF['PFC_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Points for credit_pnt']]
DS_DF['S&PWP_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['S&PWP_pnt']]

In [8]:
DS_DF['Other_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Other_pnt']]
DS_DF['Helloworld_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Helloworld_pnt']]
DS_DF['FX_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['FX_pnt']]

In [9]:
DS_DF['Webjet_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Webjet_pnt']]
DS_DF['Hotel_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Hotels_pnt']]
DS_DF['Airlocal_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Air - local_pnt']]

In [10]:
DS_DF['Ent_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Entertainment_pnt']]
DS_DF['AirLocal_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Air - Local_pnt']]
DS_DF['AmexTravel_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Amex Travel Online_pnt']]

In [11]:
DS_DF['RewardsCon_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Rewards Conceirge_pnt']]
DS_DF['QFF_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['QFF_pnt']]
DS_DF['Donations_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Donations_pnt']]
DS_DF['Safekey_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Safekey_pnt']]

In [12]:
cc=['PWP_flag','VFF_flag','AirForeign_flag','GiftCard_flag','PFC_flag','S&PWP_flag','Other_flag','Helloworld_flag',
 'FX_flag','Webjet_flag','Hotel_flag','Airlocal_flag','Ent_flag','AirLocal_flag','AmexTravel_flag','RewardsCon_flag',
 'QFF_flag','Donations_flag','Safekey_flag']

In [13]:
DS_DF['all_MR_flag']=DS_DF[cc].sum(axis=1)

In [14]:
DS_DF['retail_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Retail_Spend_ratio']]

In [15]:
DS_DF['Travel_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Travel_Spend_ratio']]

In [16]:
DS_DF['Utlities_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Utilities_Spend_ratio']]

In [17]:
DS_DF['ProfSer_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['ProfSer_Spend_ratio']]

In [18]:
DS_DF['Ent_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['Entertainment_Spend_ratio']]

In [19]:
DS_DF['B2B_flag']=[0 if ((str(x)=='nan')|(x==0)) else 1 for x in DS_DF['B2B_Spend_ratio']]

In [20]:
DS_DF['all_flag']=DS_DF[['retail_flag','Travel_flag','Utlities_flag','ProfSer_flag','Ent_flag','B2B_flag']].sum(axis=1)

In [21]:
pd.options.display.float_format = '{:,.2f}'.format
DS_DF[DS_DF['Churned']==0].groupby('acct_spend_decile').agg({'LAV_ID1':'count','acct_spend_lcl':'mean','Retail_Spend_ratio':'mean','Travel_Spend_ratio':'mean','Utilities_Spend_ratio':'mean','ProfSer_Spend_ratio':'mean','Entertainment_Spend_ratio':'mean','B2B_Spend_ratio':'mean'})

,LAV_ID1,acct_spend_lcl,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio
acct_spend_decile,,,,,,,,
-1,86916,0.00,0.70,0.13,0.09,0.03,0.05,0.01
1,45314,6.30,0.65,0.13,0.12,0.04,0.05,0.00
2,57511,32.85,0.57,0.16,0.13,0.08,0.05,0.01
3,64656,75.39,0.55,0.18,0.15,0.06,0.06,0.01
4,69137,140.05,0.52,0.21,0.15,0.06,0.05,0.01
5,71806,241.69,0.49,0.22,0.15,0.07,0.05,0.01
6,74500,405.22,0.47,0.23,0.15,0.08,0.05,0.01
7,77589,676.86,0.46,0.24,0.16,0.08,0.05,0.01
8,81461,"1,149.44",0.45,0.24,0.17,0.09,0.05,0.01


In [22]:
pd.options.display.float_format = '{:,.2f}'.format
DS_DF[DS_DF['Churned']==0].groupby('acct_spend_decile').agg({'LAV_ID1':'count','acct_spend_lcl':'mean','roc_cnt':'mean','retail_flag':'sum','Travel_flag':'sum','Utlities_flag':'sum','ProfSer_flag':'sum','Ent_flag':'sum','B2B_flag':'sum','all_flag':'mean'})
#0000

,LAV_ID1,acct_spend_lcl,roc_cnt,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,Ent_flag,B2B_flag,all_flag
acct_spend_decile,,,,,,,,,,
-1,86916,0.00,0.00,5261,1690,2270,861,1704,164,0.14
1,45314,6.30,0.19,22243,6035,10565,3033,6191,273,1.07
2,57511,32.85,0.54,38234,17670,23426,11105,15822,1122,1.87
3,64656,75.39,1.02,51071,30691,35990,18470,26022,2298,2.54
4,69137,140.05,1.70,57814,39887,43620,25931,32094,3745,2.94
5,71806,241.69,2.73,62954,47859,49917,33845,38408,5515,3.32
6,74500,405.22,4.39,67597,55411,56808,41950,44901,7916,3.69
7,77589,676.86,7.28,72924,63304,64951,51628,53010,10719,4.08
8,81461,"1,149.44",12.08,78760,72259,73668,63176,63645,15316,4.50


In [43]:
pd.options.display.float_format = '{:,.2f}'.format
DS_DF[(DS_DF['Churned']==0)&(DS_DF['MR_Yes']==1)].groupby('acct_spend_decile').agg({'LAV_ID1':'count','rdm_pntsum':'mean','rdm_cntsum':'mean','all_MR_flag':'mean'})
#0000

,LAV_ID1,rdm_pntsum,rdm_cntsum,all_MR_flag
acct_spend_decile,,,,
-1,3211,"50,243.76",1.65,1.14
1,8186,"25,226.12",1.52,1.16
2,11738,"23,573.66",1.48,1.15
3,15086,"24,371.10",1.47,1.15
4,18717,"24,092.65",1.46,1.17
5,22699,"24,772.32",1.45,1.21
6,26659,"28,102.96",1.48,1.27
7,31489,"31,776.44",1.51,1.34
8,37281,"38,838.57",1.55,1.39


In [24]:
DS_DF.head()

,Unnamed: 0,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag,all_MR_flag,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,B2B_flag,all_flag
0,0,LAV000000000000001,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"33,468.50",0,0,1,nan,nan,-1,-1,-1,-1,-1,-1,-1.00,-1.00,-1.00,4,7,3,9.00,"2,014.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,LAV000000000000002,119.65,1.32,0.00,0.00,0.00,37.00,1.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Free For Life / No Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20,0,2,4,2,1.12,1.62,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,1,0,1.44,0.00,-1,-1,-1,5,9,4,1.00,1.00,1.00,-1,-1,-1,nan,nan,140.42,0.00,0.00,0.00,0.00,0.00,1.41,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,2,LAV000000000000003,70.95,0.68,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2015-11-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,192,8.15-20,0,1,3,2,2.64,0.45,0.09,2.00,"30,712.00","30,712.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"14,422.27",1,1,1,0.17,0.20,4,7,3,2,2,1,3.00,4.00,2.00,3,5,3,11.00,"2,015.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,LAV000000000000004,"4,073.70",57.38,0.00,0.00,0.00,0.00,36.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20,0,4,10,5,4.42,1.64,0.03,1.00,"2,642.00",nan,"2,642.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"65,627.89",1,1,1,0.3

In [33]:
DS_DF['roc_cnt'][DS_DF['acct_spend_zero_flag']==1]=0

In [39]:
DS_DF[DS_DF['acct_spend_zero_flag']==0]['roc_cnt'].sum()

7416406.3243

In [36]:
DS_DF['acct_roc_decile']=0

In [40]:
DS_DF['acct_roc_decile'][DS_DF['acct_spend_zero_flag']==0]=pd.qcut(DS_DF[DS_DF['acct_spend_zero_flag']==0]['roc_cnt'], 10, labels=False) + 1

In [42]:
DS_DF['acct_roc_decile'][DS_DF['acct_spend_zero_flag']==1]=-1

In [44]:
pd.options.display.float_format = '{:,.2f}'.format
DS_DF[DS_DF['Churned']==0].groupby('acct_roc_decile').agg({'LAV_ID1':'count','acct_spend_lcl':'mean','roc_cnt':'mean','retail_flag':'sum','Travel_flag':'sum','Utlities_flag':'sum','ProfSer_flag':'sum','Ent_flag':'sum','B2B_flag':'sum','all_flag':'mean'})
#0000

,LAV_ID1,acct_spend_lcl,roc_cnt,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,Ent_flag,B2B_flag,all_flag
acct_roc_decile,,,,,,,,,,
-1,86916,0.00,0.00,5261,1690,2270,861,1704,164,0.14
1,51082,38.47,0.07,21564,9145,8402,3180,4853,548,0.93
2,59343,99.51,0.28,39171,21031,21593,9717,14438,1557,1.81
3,62705,184.33,0.68,49186,31488,32121,17353,23948,2828,2.50
4,68618,270.97,1.24,56257,38413,40048,26623,29658,3919,2.84
5,71446,417.96,2.11,63909,47656,50271,34427,37967,5982,3.36
6,73752,654.88,3.52,68850,55879,59253,43043,46406,8545,3.82
7,75716,"1,014.01",5.94,72912,62343,66931,51308,54219,11841,4.22
8,79375,"1,618.13",10.24,77864,69522,75046,61639,63952,16264,4.59


In [45]:
pd.options.display.float_format = '{:,.2f}'.format
DS_DF[(DS_DF['Churned']==0)&(DS_DF['MR_Yes']==1)].groupby('acct_roc_decile').agg({'LAV_ID1':'count','rdm_pntsum':'mean','rdm_cntsum':'mean','all_MR_flag':'mean'})
#0000

,LAV_ID1,rdm_pntsum,rdm_cntsum,all_MR_flag
acct_roc_decile,,,,
-1,3211,"50,243.76",1.65,1.14
1,7614,"28,514.69",1.46,1.13
2,13799,"28,407.32",1.48,1.15
3,15694,"28,829.38",1.44,1.15
4,18407,"30,042.77",1.42,1.17
5,22289,"31,309.03",1.45,1.21
6,26267,"35,174.85",1.47,1.26
7,30868,"41,132.41",1.49,1.32
8,36654,"49,341.97",1.53,1.38


In [29]:
#DS_DF.isnull().sum()

In [53]:
DS_DF['SuppStatus_1'].value_counts()

0     937610
37    367974
Name: SuppStatus_1, dtype: int64

In [65]:
DS_DF.shape

(1305584, 142)

In [86]:
DS_DF.head()

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag,all_MR_flag,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,B2B_flag,all_flag,acct_roc_decile,Churned_month,Churned_year,churne_month_year
0,LAV000000000000001,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"33,468.50",0,0,1,nan,nan,-1,-1,-1,-1,-1,-1,-1.00,-1.00,-1.00,4,7,3,9.00,"2,014.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,09,2014,092014
1,LAV000000000000002,119.65,1.32,0.00,0.00,0.00,37.00,1.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Free For Life / No Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20,0,2,4,2,1.12,1.62,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,1,0,1.44,0.00,-1,-1,-1,5,9,4,1.00,1.00,1.00,-1,-1,-1,nan,nan,140.42,0.00,0.00,0.00,0.00,0.00,1.41,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,4,01,2100,012100
2,LAV000000000000003,70.95,0.68,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2015-11-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,192,8.15-20,0,1,3,2,2.64,0.45,0.09,2.00,"30,712.00","30,712.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"14,422.27",1,1,1,0.17,0.20,4,7,3,2,2,1,3.00,4.00,2.00,3,5,3,11.00,"2,015.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3,11,2015,112015
3,LAV000000000000004,"4,073.70",57.38,0.00,0.00,0.00,0.00,36.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20,0,4,10,5,4.42,1.64,0.03,1.00,"2,642.00",nan,"2,642.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,

In [67]:
DS_DF.to_csv('/Users/sri.t/Desktop/amex_churners_estimates_4.csv',header=True,sep=',',encoding='utf-8')

In [47]:
#DS_DF_1=pd.read_csv('/Users/sri.t/Desktop/Spend_By_Industry_3_year_with_ratio.csv',sep=',')

In [58]:
#DS_DF_1.shape

In [57]:
#DS_DF_1.head()

In [6]:
DS_DF.drop(columns=['Unnamed: 0','Unnamed: 0.1'],axis=1,inplace=True)

In [56]:
DS_DF.head()

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag,all_MR_flag,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,B2B_flag,all_flag,acct_roc_decile,Churned_month,Churned_year,churne_month_year,acct_spend_decile_1,Adv_Publ,Air_Dom,Air_Intl,Air_Net,ATO,Car_Rental,Computer_Software,Consumables,Costco,Courier_Freight,Cruise,Dining,DJ_Prop_BB,DJ_GPC_BB,Education,Entertainment,Everything_Else,Financial_Services,Gas,Healthcare,Insurance,Lodging,Other_Air,Other_Gov,Other_Travel,Outbound,Personal_Services,QF_Air,Retail_DepStores,SBS_B2B,Supermarkets,Coles,Woolworths,Taxi_Limo,Telecomms,Tolls_Private,Transportation,Travel_Agents,Utilities,VA_Air,total,Adv_Publ_ratio,Air_Dom_ratio,Air_Intl_ratio,Air_Net_ratio,ATO_ratio,Car_Rental_ratio,Computer_Software_ratio,Consumables_ratio,Costco_ratio,Courier_Freight_ratio,Cruise_ratio,Dining_ratio,DJ_Prop_BB_ratio,DJ_GPC_BB_ratio,Education_ratio,Entertainment_ratio,Everything_Else_ratio,Financial_Services_ratio,Gas_ratio,Healthcare_ratio,Insurance_ratio,Lodging_ratio,Other_Air_ratio,Other_Gov_ratio,Other_Travel_ratio,Outbound_ratio,Personal_Services_ratio,QF_Air_ratio,Retail_DepStores_ratio,SBS_B2B_ratio,Supermarkets_ratio,Coles_ratio,Woolworths_ratio,Taxi_Limo_ratio,Telecomms_ratio,Tolls_Private_ratio,Transportation_ratio,Travel_Agents_ratio,Utilities_ratio,VA_Air_ratio,count
0,LAV000000000000001,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.00000000,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"33,468.50000000",0,0,1,nan,nan,-1,-1,-1,-1,-1,-1,-1.00000000,-1.00000000,-1.00000000,4,7,3,9.00000000,"2,014.00000000",0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,nan,nan,nan,nan,nan,nan,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,09,2014,201409,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan

In [8]:
DS_DF['Churned_month']=[x[5:7] for x in DS_DF['cancel_dt']]

In [9]:
DS_DF['Churned_year']=[x[0:4] for x in DS_DF['cancel_dt']]

In [10]:
DS_DF['churne_month_year']=[a+c for c, a in zip(DS_DF['Churned_month'], DS_DF['Churned_year'])]

In [11]:
y=DS_DF[DS_DF['Churned']==1].pivot_table(index=['churne_month_year'], columns=['product_description'], values=['LAV_ID1'],aggfunc='count').reset_index()

In [103]:
y=y.fillna(value=0)

In [105]:
#y

In [108]:
y=DS_DF[DS_DF['Churned']==1].pivot_table(index=['churne_month_year'], columns=['detailed_fee_type'], values=['LAV_ID1'],aggfunc='count').reset_index().fillna(value=0)

In [12]:
#y

In [14]:
pd.options.display.float_format = '{:,.2f}'.format
DS_DF[DS_DF['Churned']==1].groupby('SuppStatus_1').agg({'LAV_ID1':'count','acct_spend_lcl':'mean','roc_cnt':'mean','acct_supp_spend_lcl':'mean','roc_supp_cnt':'mean','retail_flag':'sum','Travel_flag':'sum','Utlities_flag':'sum','ProfSer_flag':'sum','Ent_flag':'sum','B2B_flag':'sum','all_flag':'mean'})
#0000

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,Ent_flag,B2B_flag,all_flag
SuppStatus_1,,,,,,,,,,,,
0,506260,175.90,1.14,33.37,0.21,47470,28671,34306,19239,25296,3617,0.31


In [57]:
pd.options.display.float_format = '{:,.2f}'.format
DS_DF[DS_DF['Churned']==0].groupby('acct_spend_decile').agg({'LAV_ID1':'count','acct_spend_lcl':['mean','std'],'roc_cnt':['mean','std'],'acct_supp_spend_lcl':['mean','std'],'roc_supp_cnt':['mean','std'],'count':['mean','std'],'all_MR_flag':['mean','std']})
#000

LAV_ID1 acct_spend_lcl           roc_cnt        \
                    count           mean       std    mean   std   
acct_spend_decile                                                  
-1                  86916           0.00      0.00    0.00  0.00   
 1                  45314           6.30      4.63    0.19  3.05   
 2                  57511          32.85     10.59    0.54  0.53   
 3                  64656          75.39     14.76    1.02  0.89   
 4                  69137         140.05     23.04    1.70  1.38   
 5                  71806         241.69     36.64    2.73  2.10   
 6                  74500         405.22     59.68    4.39  3.27   
 7                  77589         676.86    100.71    7.28  5.10   
 8                  81461       1,149.44    180.32   12.08  7.71   
 9                  84575       2,076.17    394.55   19.46 11.33   
 10                 85859       8,933.30 17,335.79   33.78 28.55   

                  acct_supp_spend_lcl           roc_supp_cnt       count       \
                                 mean       std         mean   std  mean  std   
acct_spend_decile                                                               
-1                               0.00      0.08         0.00  0.00  0.26 1.24   
 1                               0.34      1.61         0.01  0.06  2.58 2.23   
 2                               1.73      6.72         0.03  0.15  4.85 3.22   
 3                               4.15     14.56         0.06  0.27  6.88 3.79   
 4                               8.89     28.12         0.12  0.45  8.58 4.25   
 5                              17.68     50.44         0.22  0.75 10.44 4.62   
 6                              33.34     87.15         0.39  1.20 12.48 4.83   
 7                              66.26    154.29         0.81  2.24 14.79 4.90   
 8                             140.24    279.43         1.72  3.90 17.39 4.76   
 9                             369.41    557.79         4.33  7.18 20.20 4.60   
 10                          2,566.20 10,474.55        12.15 17.71 23.86 5.01   

                  all_MR_flag       
                         mean  std  
acct_spend_decile                   
-1                       0.04 0.23  
 1                       0.21 0.48  
 2                       0.23 0.49  
 3                       0.27 0.52  
 4                       0.32 0.56  
 5                       0.38 0.62  
 6                       0.45 0.68  
 7                       0.54 0.75  
 8                       0.63 0.81  
 9                       0.75 0.88  
 10                      0.94 0.98

In [38]:
DS_DF['acct_spend_decile_1']=0

In [39]:
DS_DF['acct_spend_decile_1'][(DS_DF['acct_spend_zero_flag']==0)&(DS_DF['Churned']==0)]=pd.qcut(DS_DF[(DS_DF['acct_spend_zero_flag']==0)&(DS_DF['Churned']==0)]['acct_spend_lcl'], 10, labels=False) + 1

In [40]:
DS_DF['acct_spend_decile_1'][(DS_DF['acct_spend_zero_flag']==1)&(DS_DF['Churned']==0)]=-1

In [41]:
DS_DF['acct_spend_decile_1'][(DS_DF['Churned']==1)]=-2

In [44]:
#DS_DF[DS_DF['acct_spend_decile_1']>0]['acct_spend_decile_1'].shape

(712408,)

In [46]:
DS_DF.shape

(1305584, 146)

In [45]:
DS_DF.head()

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag,all_MR_flag,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,B2B_flag,all_flag,acct_roc_decile,Churned_month,Churned_year,churne_month_year,acct_spend_decile_1
0,LAV000000000000001,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"33,468.50",0,0,1,nan,nan,-1,-1,-1,-1,-1,-1,-1.00,-1.00,-1.00,4,7,3,9.00,"2,014.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,09,2014,201409,-2
1,LAV000000000000002,119.65,1.32,0.00,0.00,0.00,37.00,1.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Free For Life / No Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20,0,2,4,2,1.12,1.62,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,1,0,1.44,0.00,-1,-1,-1,5,9,4,1.00,1.00,1.00,-1,-1,-1,nan,nan,140.42,0.00,0.00,0.00,0.00,0.00,1.41,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,4,01,2100,210001,3
2,LAV000000000000003,70.95,0.68,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2015-11-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,192,8.15-20,0,1,3,2,2.64,0.45,0.09,2.00,"30,712.00","30,712.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"14,422.27",1,1,1,0.17,0.20,4,7,3,2,2,1,3.00,4.00,2.00,3,5,3,11.00,"2,015.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3,11,2015,201511,-2
3,LAV000000000000004,"4,073.70",57.38,0.00,0.00,0.00,0.00,36.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2100-01-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,0,11,1999,1999-11-01,216,8.15-20,0,4,10,5,4.42,1.64,0.03,1.00,"2,642.00",nan,"2,642.00",nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,

In [15]:
DF1=pd.read_csv('/Users/sri.t/Desktop/Spend_By_Industry.csv',sep=',')

In [16]:
DF1.head()

,Unnamed: 0,LAV_ID1,Adv_Publ,Air_Dom,Air_Intl,Air_Net,ATO,Car_Rental,Computer_Software,Consumables,Costco,Courier_Freight,Cruise,Dining,DJ_Prop_BB,DJ_GPC_BB,Education,Entertainment,Everything_Else,Financial_Services,Gas,Healthcare,Insurance,Lodging,Other_Air,Other_Gov,Other_Travel,Outbound,Personal_Services,QF_Air,Retail_DepStores,SBS_B2B,Supermarkets,Coles,Woolworths,Taxi_Limo,Telecomms,Tolls_Private,Transportation,Travel_Agents,Utilities,VA_Air,total,Adv_Publ_ratio,Air_Dom_ratio,Air_Intl_ratio,Air_Net_ratio,ATO_ratio,Car_Rental_ratio,Computer_Software_ratio,Consumables_ratio,Costco_ratio,Courier_Freight_ratio,Cruise_ratio,Dining_ratio,DJ_Prop_BB_ratio,DJ_GPC_BB_ratio,Education_ratio,Entertainment_ratio,Everything_Else_ratio,Financial_Services_ratio,Gas_ratio,Healthcare_ratio,Insurance_ratio,Lodging_ratio,Other_Air_ratio,Other_Gov_ratio,Other_Travel_ratio,Outbound_ratio,Personal_Services_ratio,QF_Air_ratio,Retail_DepStores_ratio,SBS_B2B_ratio,Supermarkets_ratio,Coles_ratio,Woolworths_ratio,Taxi_Limo_ratio,Telecomms_ratio,Tolls_Private_ratio,Transportation_ratio,Travel_Agents_ratio,Utilities_ratio,VA_Air_ratio,count
0,0,LAV000000000000001,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0
1,1,LAV000000000000002,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,132.62,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.42,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.56,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3
2,2,LAV000000000000003,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,35.52,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.79,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3
3,3,LAV000000000000004,0,0,1,1,0,1,1,1,0,0,0,1,1,0,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,1,0,1,"4,487.20",0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.06,0.00,0.00,0.00,0.00,0.04,0.00,0.26,0.00,0.00,0.02,0.00,0.00,0.02,0.13,0.00,0.00,0.19,0.00,0.01,0.04,0.09,0.00,0.04,0.00,0.01,0.00,0.00,0.00,27
4,4,LAV000000000000005,1,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,1,1,1,1,0,1,0,0,0,"3,287.14",0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.11,0.00,0.00,0.00,0.04,0.00,0.00,0.05,0.00,0.01,0.00,0.20,0.00,0.02,0.03,0.00,0.00,0.00,0.09,0.00,0.00,0.36,0.00,0.00,0.00,0.02,0.00,0.04,0.00,0.01,0.00,0.00,0.00,21


In [19]:
DF1.drop(columns='Unnamed: 0',axis=1,inplace=True)

In [47]:
DF1.shape

(1305584, 83)

In [48]:
DS_DF=DS_DF.merge(DF1,on='LAV_ID1',how='inner')

In [49]:
DS_DF.shape

(1305584, 228)

In [55]:
pd.options.display.float_format = '{:,.8f}'.format
DS_DF[DS_DF['Churned']==0].groupby('acct_spend_decile')[col_ind].agg('mean')
#000

,Adv_Publ_ratio,Air_Dom_ratio,Air_Intl_ratio,Air_Net_ratio,ATO_ratio,Car_Rental_ratio,Computer_Software_ratio,Consumables_ratio,Costco_ratio,Courier_Freight_ratio,Cruise_ratio,Dining_ratio,DJ_Prop_BB_ratio,DJ_GPC_BB_ratio,Education_ratio,Entertainment_ratio,Everything_Else_ratio,Financial_Services_ratio,Gas_ratio,Healthcare_ratio,Insurance_ratio,Lodging_ratio,Other_Air_ratio,Other_Gov_ratio,Other_Travel_ratio,Outbound_ratio,Personal_Services_ratio,QF_Air_ratio,Retail_DepStores_ratio,SBS_B2B_ratio,Supermarkets_ratio,Coles_ratio,Woolworths_ratio,Taxi_Limo_ratio,Telecomms_ratio,Tolls_Private_ratio,Transportation_ratio,Travel_Agents_ratio,Utilities_ratio,VA_Air_ratio
acct_spend_decile,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-1,0.00196976,0.00001355,0.00590257,0.00220684,0.00578758,0.00148389,0.00016097,0.01766071,0.00000000,0.00001313,0.00065168,0.02730937,0.00485051,0.04135310,0.00259686,0.00642428,0.54464457,0.00000000,0.04231770,0.00604521,0.00925846,0.00753357,0.00138617,0.00682136,0.00469036,0.03913397,0.00410230,0.00957185,0.12402858,0.00005913,0.01017803,0.01918882,0.01965600,0.00299204,0.00659423,0.00039328,0.00283394,0.01381937,0.00224648,0.00411979
1,0.00332338,0.00001318,0.00384733,0.00081137,0.00062711,0.00442435,0.00119573,0.02679181,0.00061643,0.00009947,0.00048419,0.03872813,0.00710803,0.16786512,0.00193055,0.01637410,0.15458877,0.00000000,0.08628985,0.00618596,0.02853391,0.00983728,0.00088011,0.01385648,0.00389189,0.07850793,0.01201916,0.00547584,0.18031089,0.00009673,0.01762206,0.04128473,0.03492597,0.00644682,0.01805973,0.00649228,0.00107981,0.00994956,0.00226780,0.00715619
2,0.00459132,0.00002818,0.00950813,0.00201479,0.00269897,0.00552448,0.00130807,0.02359517,0.00099021,0.00013589,0.00128478,0.03352180,0.00611373,0.17172992,0.00400518,0.01926494,0.06858127,0.00000000,0.08179774,0.00938794,0.05798350,0.01568701,0.00223967,0.01466302,0.00825301,0.07693501,0.00779091,0.01329385,0.20909944,0.00005562,0.01776547,0.03427915,0.03254078,0.00447500,0.02336662,0.00505729,0.00392196,0.01246979,0.00625097,0.00778941
3,0.00380296,0.00005435,0.01316897,0.00425783,0.00407529,0.00490957,0.00149159,0.02356900,0.00088098,0.00020571,0.00193396,0.03574258,0.00582621,0.13040100,0.00528242,0.01667347,0.06057745,0.00000000,0.08764560,0.01074854,0.04000160,0.01747145,0.00305435,0.01509599,0.01156324,0.07931349,0.00770053,0.01687460,0.22877569,0.00004822,0.01894258,0.03516973,0.03483614,0.00536237,0.03007064,0.00384251,0.00507836,0.01952014,0.00727083,0.00876005
4,0.00377580,0.00006130,0.01508240,0.00594620,0.00544515,0.00474487,0.00123651,0.02297308,0.00110168,0.00021551,0.00227815,0.03576842,0.00562936,0.11303399,0.00588905,0.01555851,0.04791060,0.00000075,0.08805771,0.01246017,0.03894218,0.01933224,0.00322293,0.01496853,0.01364702,0.08569452,0.00816189,0.01878656,0.23119219,0.00005701,0.01804259,0.03539723,0.03500317,0.00588732,0.03348792,0.00381752,0.00629724,0.02290146,0.00813330,0.00985797
5,0.00353588,0.00005766,0.01606672,0.00721835,0.00683130,0.00506716,0.00119762,0.02262086,0.00133402,0.00030252,0.00240858,0.03653674,0.00554408,0.09358755,0.00687946,0.01523644,0.04220275,0.00000128,0.08982494,0.01394994,0.04157881,0.02043106,0.00404330,0.01559833,0.01492620,0.09034198,0.00873991,0.02072105,0.23128773,0.00004283,0.01792406,0.03625979,0.03571427,0.00625883,0.02936339,0.00366413,0.00782135,0.02646409,0.00812916,0.01028587
6,0.00367215,0.00006934,0.01672968,0.00771244,0.00900014,0.00485468,0.00113522,0.02273212,0.00165921,0.00025383,0.00249508,0.03767836,0.00573537,0.06894846,0.00831336,0.01423311,0.03697325,0.00000000,0.09354742,0.01463805,0.04630082,0.02141533,0.00453557,0.01522294,0.01615055,0.09483173,0.00893850,0.02284268,0.23291634,0.00005233,0.01875727,0.03949058,0.03844298,0.00666743,0.02528544,0.00314043,0.00810642,0.02783859,0.00798121,0.01070158
7,0.00355254,0.00005743,0.01638229,0.00838569,0.00976829,0.00475316,0.00117440,0.02331079,0.00206948,0.00033497,0.00288171,0.03795553,0.00595423,0.04576927,0.00902278,0.013420

In [53]:
#DF1.columns

In [52]:
col_ind=['Adv_Publ_ratio',
       'Air_Dom_ratio', 'Air_Intl_ratio', 'Air_Net_ratio', 'ATO_ratio',
       'Car_Rental_ratio', 'Computer_Software_ratio', 'Consumables_ratio',
       'Costco_ratio', 'Courier_Freight_ratio', 'Cruise_ratio', 'Dining_ratio',
       'DJ_Prop_BB_ratio', 'DJ_GPC_BB_ratio', 'Education_ratio',
       'Entertainment_ratio', 'Everything_Else_ratio',
       'Financial_Services_ratio', 'Gas_ratio', 'Healthcare_ratio',
       'Insurance_ratio', 'Lodging_ratio', 'Other_Air_ratio',
       'Other_Gov_ratio', 'Other_Travel_ratio', 'Outbound_ratio',
       'Personal_Services_ratio', 'QF_Air_ratio', 'Retail_DepStores_ratio',
       'SBS_B2B_ratio', 'Supermarkets_ratio', 'Coles_ratio',
       'Woolworths_ratio', 'Taxi_Limo_ratio', 'Telecomms_ratio',
       'Tolls_Private_ratio', 'Transportation_ratio', 'Travel_Agents_ratio',
       'Utilities_ratio', 'VA_Air_ratio']

In [58]:
del DF1

In [59]:
DS_DF['supp_status']=[0 if x<=0 else 1 for x in DS_DF['acct_supp_spend_lcl']]

In [61]:
DS_DF[DS_DF['Churned']==1]['supp_status'].value_counts()

0    472126
1     34134
Name: supp_status, dtype: int64

In [64]:
pd.crosstab(DS_DF[DS_DF['Churned']==1]['SuppStatus_1'],DS_DF[DS_DF['Churned']==1]['supp_status'])

supp_status,0,1
SuppStatus_1,,
0,472126,34134


In [65]:
pd.crosstab(DS_DF[DS_DF['Churned']==0]['SuppStatus_1'],DS_DF[DS_DF['Churned']==0]['supp_status'])

supp_status,0,1
SuppStatus_1,,
0,425327,6023
37,161346,206628


In [66]:
DS_DF.shape

(1305584, 229)

In [68]:
pd.options.display.float_format = '{:,.2f}'.format
DS_DF[DS_DF['Churned']==0].groupby('acct_spend_decile_1').agg({'LAV_ID1':'count','acct_spend_lcl':['mean','std'],'roc_cnt':['mean','std'],'acct_supp_spend_lcl':['mean','std'],'roc_supp_cnt':['mean','std'],'count':['mean','std'],'all_MR_flag':['mean','std']})
#000

LAV_ID1 acct_spend_lcl           roc_cnt        \
                      count           mean       std    mean   std   
acct_spend_decile_1                                                  
-1                    86916           0.00      0.00    0.00  0.00   
 1                    71246          12.29      9.11    0.27  2.44   
 2                    71240          54.63     14.12    0.79  0.72   
 3                    71253         114.19     20.80    1.44  1.19   
 4                    71233         203.41     31.61    2.34  1.83   
 5                    71234         340.40     48.68    3.74  2.80   
 6                    71245         553.49     76.14    5.93  4.29   
 7                    71237         889.07    120.85    9.54  6.39   
 8                    71241       1,433.19    201.64   14.63  8.85   
 9                    71238       2,456.17    433.98   21.93 12.49   
 10                   71241      10,128.13 18,809.77   35.43 30.42   

                    acct_supp_spend_lcl           roc_supp_cnt       count  \
                                   mean       std         mean   std  mean   
acct_spend_decile_1                                                          
-1                                 0.00      0.08         0.00  0.00  0.26   
 1                                 0.64      3.04         0.01  0.09  3.18   
 2                                 2.88     10.62         0.04  0.21  6.03   
 3                                 6.94     22.75         0.09  0.37  7.98   
 4                                14.29     42.09         0.18  0.65  9.82   
 5                                27.06     72.64         0.32  1.02 11.75   
 6                                50.15    123.33         0.60  1.75 13.85   
 7                                96.18    207.87         1.18  2.96 16.12   
 8                               197.23    358.46         2.41  4.95 18.48   
 9                               489.55    673.06         5.53  8.31 21.01   
 10                            2,945.65 11,455.87        13.08 18.79 24.20   

                         all_MR_flag       
                     std        mean  std  
acct_spend_decile_1                        
-1                  1.24        0.04 0.23  
 1                  2.63        0.22 0.48  
 2                  3.58        0.25 0.50  
 3                  4.10        0.30 0.55  
 4                  4.50        0.36 0.60  
 5                  4.79        0.43 0.66  
 6                  4.89        0.50 0.73  
 7                  4.83        0.59 0.79  
 8                  4.69        0.68 0.84  
 9                  4.53        0.78 0.88  
 10                 5.05        0.97 0.99

In [85]:
DS_DF.head()

,Unnamed: 0,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag,all_MR_flag,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,B2B_flag,all_flag,acct_roc_decile,Churned_month,Churned_year,churne_month_year,acct_spend_decile_1,Adv_Publ,Air_Dom,Air_Intl,Air_Net,ATO,Car_Rental,Computer_Software,Consumables,Costco,Courier_Freight,Cruise,Dining,DJ_Prop_BB,DJ_GPC_BB,Education,Entertainment,Everything_Else,Financial_Services,Gas,Healthcare,Insurance,Lodging,Other_Air,Other_Gov,Other_Travel,Outbound,Personal_Services,QF_Air,Retail_DepStores,SBS_B2B,Supermarkets,Coles,Woolworths,Taxi_Limo,Telecomms,Tolls_Private,Transportation,Travel_Agents,Utilities,VA_Air,total,Adv_Publ_ratio,Air_Dom_ratio,Air_Intl_ratio,Air_Net_ratio,ATO_ratio,Car_Rental_ratio,Computer_Software_ratio,Consumables_ratio,Costco_ratio,Courier_Freight_ratio,Cruise_ratio,Dining_ratio,DJ_Prop_BB_ratio,DJ_GPC_BB_ratio,Education_ratio,Entertainment_ratio,Everything_Else_ratio,Financial_Services_ratio,Gas_ratio,Healthcare_ratio,Insurance_ratio,Lodging_ratio,Other_Air_ratio,Other_Gov_ratio,Other_Travel_ratio,Outbound_ratio,Personal_Services_ratio,QF_Air_ratio,Retail_DepStores_ratio,SBS_B2B_ratio,Supermarkets_ratio,Coles_ratio,Woolworths_ratio,Taxi_Limo_ratio,Telecomms_ratio,Tolls_Private_ratio,Transportation_ratio,Travel_Agents_ratio,Utilities_ratio,VA_Air_ratio,count,supp_status
0,0,LAV000000000000001,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0,0.00000000,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"33,468.50000000",0,0,1,nan,nan,-1,-1,-1,-1,-1,-1,-1.00000000,-1.00000000,-1.00000000,4,7,3,9.00000000,"2,014.00000000",0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,nan,nan,nan,nan,nan,nan,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,9,2014,201409,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan

In [86]:
DS_DF.drop(columns='Unnamed: 0',inplace=True,axis=1)

In [124]:
df=pd.read_csv("/Users/sri.t/Desktop/AMEX1/Churn_Data/cluster_data_with_churn.csv",sep=',')
df.shape

(811094, 305)

In [146]:
df.head()

,LAV_ID1,Gender,IncomeCat,ACQ_TYPE,acq_chn,detailed_fee_type,product_description,days_active,estab_dt,MR_Source_Code,Income,Dob,PostCode,CHANNEL,optout,prod_tier,SuppStatus,AgeCat,myca_app_ind_1,acct_myca_active_1,myca_active_mob_1,acct_spend_lcl Q1,acct_spend_lcl Q2,acct_spend_lcl Q3,acct_spend_lcl Q4,acct_supp_spend_lcl Q1,acct_supp_spend_lcl Q2,acct_supp_spend_lcl Q3,acct_supp_spend_lcl Q4,roc_cnt Q1,roc_cnt Q2,roc_cnt Q3,roc_cnt Q4,roc_supp_cnt Q1,roc_supp_cnt Q2,roc_supp_cnt Q3,roc_supp_cnt Q4,Mean_apps_logs_month,BtStatus_1,Mean_act_supp_month,curr_bal_amt Q1,curr_bal_amt Q2,curr_bal_amt Q3,curr_bal_amt Q4,LOC Q1,LOC Q2,LOC Q3,LOC Q4,Mean_myca_logs_month,revolving_ind,total_apps_count_bin,days_active_bin,active_supps_log_bin,Retail_spend_Q1,Retail_spend_Q2,Retail_spend_Q3,Retail_spend_Q4,Travel_spend_Q1,Travel_spend_Q2,Travel_spend_Q3,Travel_spend_Q4,Util_spend_Q1,Util_spend_Q2,Util_spend_Q3,Util_spend_Q4,PrSr_spend_Q1,PrSr_spend_Q2,PrSr_spend_Q3,PrSr_spend_Q4,Ent_spend_Q1,Ent_spend_Q2,Ent_spend_Q3,Ent_spend_Q4,B2B_spend_Q1,B2B_spend_Q2,B2B_spend_Q3,B2B_spend_Q4,Retail_Cnt_Q1,Retail_Cnt_Q2,Retail_Cnt_Q3,Retail_Cnt_Q4,Travel_Cnt_Q1,Travel_Cnt_Q2,Travel_Cnt_Q3,Travel_Cnt_Q4,Util_Cnt_Q1,Util_Cnt_Q2,Util_Cnt_Q3,Util_Cnt_Q4,PrSr_Cnt_Q1,PrSr_Cnt_Q2,PrSr_Cnt_Q3,PrSr_Cnt_Q4,Ent_Cnt_Q1,Ent_Cnt_Q2,Ent_Cnt_Q3,Ent_Cnt_Q4,B2B_Cnt_Q1,B2B_Cnt_Q2,B2B_Cnt_Q3,B2B_Cnt_Q4,Retail_spend_Q1_ratio,Retail_spend_Q2_ratio,Retail_spend_Q3_ratio,Retail_spend_Q4_ratio,Travel_spend_Q1_ratio,Travel_spend_Q2_ratio,Travel_spend_Q3_ratio,Travel_spend_Q4_ratio,Util_spend_Q1_ratio,Util_spend_Q2_ratio,Util_spend_Q3_ratio,Util_spend_Q4_ratio,PrSr_spend_Q1_ratio,PrSr_spend_Q2_ratio,PrSr_spend_Q3_ratio,PrSr_spend_Q4_ratio,Ent_spend_Q1_ratio,Ent_spend_Q2_ratio,Ent_spend_Q3_ratio,Ent_spend_Q4_ratio,B2B_spend_Q1_ratio,B2B_spend_Q2_ratio,B2B_spend_Q3_ratio,B2B_spend_Q4_ratio,IS_YES,Gift_pnt_Q1,Gift_pnt_Q2,Gift_pnt_Q3,Gift_pnt_Q4,PWP_pnt_Q1,PWP_pnt_Q2,PWP_pnt_Q3,PWP_pnt_Q4,PFC_pnt_Q1,PFC_pnt_Q2,PFC_pnt_Q3,PFC_pnt_Q4,VFF_pnt_Q1,VFF_pnt_Q2,VFF_pnt_Q3,VFF_pnt_Q4,SPWP_pnt_Q1,SPWP_pnt_Q2,SPWP_pnt_Q3,SPWP_pnt_Q4,Air_Oth_pnt_Q1,Air_Oth_pnt_Q2,Air_Oth_pnt_Q3,Air_Oth_pnt_Q4,Travel_pnt_Q1,Travel_pnt_Q2,Travel_pnt_Q3,Travel_pnt_Q4,Others_pnt_Q1,Others_pnt_Q2,Others_pnt_Q3,Others_pnt_Q4,PWP_cnt_Q1,PWP_cnt_Q2,PWP_cnt_Q3,PWP_cnt_Q4,Gift_cnt_Q1,Gift_cnt_Q2,Gift_cnt_Q3,Gift_cnt_Q4,PFC_cnt_Q1,PFC_cnt_Q2,PFC_cnt_Q3,PFC_cnt_Q4,VFF_cnt_Q1,VFF_cnt_Q2,VFF_cnt_Q3,VFF_cnt_Q4,SPWP_cnt_Q1,SPWP_cnt_Q2,SPWP_cnt_Q3,SPWP_cnt_Q4,Air_Oth_cnt_Q1,Air_Oth_cnt_Q2,Air_Oth_cnt_Q3,Air_Oth_cnt_Q4,Travel_cnt_Q1,Travel_cnt_Q2,Travel_cnt_Q3,Travel_cnt_Q4,Others_cnt_Q1,Others_cnt_Q2,Others_cnt_Q3,Others_cnt_Q4,PWP_RdPB_Q1,PWP_RdPB_Q2,PWP_RdPB_Q3,PWP_RdPB_Q4,Gift_RdPB_Q1,Gift_RdPB_Q2,Gift_RdPB_Q3,Gift_RdPB_Q4,PFC_RdPB_Q1,PFC_RdPB_Q2,PFC_RdPB_Q3,PFC_RdPB_Q4,VFF_RdPB_Q1,VFF_RdPB_Q2,VFF_RdPB_Q3,VFF_RdPB_Q4,SPWP_RdPB_Q1,SPWP_RdPB_Q2,SPWP_RdPB_Q3,SPWP_RdPB_Q4,Air_Oth_RdPB_Q1,Air_Oth_RdPB_Q2,Air_Oth_RdPB_Q3,Air_Oth_RdPB_Q4,Travel_RdPB_Q1,Travel_RdPB_Q2,Travel_RdPB_Q3,Travel_RdPB_Q4,Others_RdPB_Q1,Others_RdPB_Q2,Others_RdPB_Q3,Others_RdPB_Q4,RDM_pnt_Q1,RDM_pnt_Q2,RDM_pnt_Q3,RDM_pnt_Q4,RDM_cnt_Q1,RDM_cnt_Q2,RDM_cnt_Q3,RDM_cnt_Q4,All_RdPB_Q1,All_RdPB_Q2,All_RdPB_Q3,All_RdPB_Q4,MR_YES,total_sent Q1,total_sent Q2,total_sent Q3,total_sent Q4,total_opens Q1,total_opens Q2,total_opens Q3,total_opens Q4,total_clicks Q1,total_clicks Q2,total_clicks Q3,total_clicks Q4,E_O/S_Q1,E_O/S_Q2,E_O/S_Q3,E_O/S_Q4,E_C/O_Q1,E_C/O_Q2,E_C/O_Q3,E_C/O_Q4,EMAIL_YES,PIB_Q1,PIB_Q2,PIB_Q3,PIB_Q4,PIB_YES,Phone_pay_Q1,Phone_pay_Q2,Phone_pay_Q3,Phone_pay_Q4,Watch_pay_Q1,Watch_pay_Q2,Watch_pay_Q3,Watch_pay_Q4,Other_pay_Q1,Other_pay_Q2,Other_pay_Q3,Other_pay_Q4,MP_yes,FF_taken,FF_Count,FH_taken,FH_Count,MGMee_count,BONUS_POINT_CT_MGMee,MGMer_count,BONUS_POINT_CT_MGMer,cluster_label,all_zero_3year,ENROL_DATE,CM_ACCT_STAT_CD,MBR_STAT_CD,MENR_YES,# of Local Champions,LC_YES,cancel_dt,months_active,estab_dt_1,pred_prob,pred_value_0.5,pred_value_0.75,pred_

In [126]:
df.drop(columns='Unnamed: 0',inplace=True,axis=1)

In [128]:
x=df.columns

In [142]:
y=['detailed_fee_type' if i=='detailed_fee_type_x' else 'product_description' if i=='product_description_x' else 'estab_dt' if i=='estab_dt_x' else 'AgeCat' if i=='AgeCat_x' else i for i in x]
#000

In [144]:
df.columns=y

In [145]:
#df.isnull().sum()

In [73]:
pd.options.display.float_format = '{:,.8f}'.format
DS_DF.groupby('Churned')[col_ind].agg('mean')
#000

,Adv_Publ_ratio,Air_Dom_ratio,Air_Intl_ratio,Air_Net_ratio,ATO_ratio,Car_Rental_ratio,Computer_Software_ratio,Consumables_ratio,Costco_ratio,Courier_Freight_ratio,Cruise_ratio,Dining_ratio,DJ_Prop_BB_ratio,DJ_GPC_BB_ratio,Education_ratio,Entertainment_ratio,Everything_Else_ratio,Financial_Services_ratio,Gas_ratio,Healthcare_ratio,Insurance_ratio,Lodging_ratio,Other_Air_ratio,Other_Gov_ratio,Other_Travel_ratio,Outbound_ratio,Personal_Services_ratio,QF_Air_ratio,Retail_DepStores_ratio,SBS_B2B_ratio,Supermarkets_ratio,Coles_ratio,Woolworths_ratio,Taxi_Limo_ratio,Telecomms_ratio,Tolls_Private_ratio,Transportation_ratio,Travel_Agents_ratio,Utilities_ratio,VA_Air_ratio
Churned,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.00419267,0.00004923,0.01446361,0.00649527,0.01216220,0.00473124,0.00124412,0.02382267,0.00165519,0.00039333,0.00263129,0.03780967,0.00619512,0.07355745,0.00899682,0.01428236,0.05340149,0.00000037,0.09407391,0.01429222,0.04359384,0.02058240,0.00411090,0.01561314,0.01299661,0.09007254,0.00904981,0.02278241,0.22054338,0.00007957,0.01987734,0.04076491,0.04001931,0.00589983,0.02513762,0.00346154,0.00783304,0.02586525,0.00767266,0.00959367
1,0.00391715,0.00004476,0.01209329,0.00473300,0.00677513,0.00533980,0.00145907,0.02439614,0.00286231,0.00032491,0.00170540,0.03950105,0.00628706,0.10964245,0.00606509,0.01350075,0.08482569,0.00000132,0.09564813,0.01118156,0.03805208,0.01917894,0.00381397,0.01460692,0.00973825,0.05577886,0.00951539,0.01596117,0.22235472,0.00007245,0.01908407,0.04487712,0.03867578,0.00654317,0.02887503,0.00414783,0.00515192,0.01706268,0.00672533,0.00948024


In [89]:
ccc=['LAV_ID1','cluster_label','all_zero_3year']

In [90]:
df1=df[ccc]

In [91]:
del df

In [92]:
df1.shape

(811094, 3)

In [93]:
DS_DF.shape

(1305584, 229)

In [94]:
DS_DF=DS_DF.merge(df1,on='LAV_ID1',how='left')

In [95]:
DS_DF.shape

(1305584, 231)

In [118]:
#DS_DF.isnull().sum()

In [98]:
DS_DF['cluster_label'][DS_DF['Churned']==1]=998

In [99]:
DS_DF['cluster_label'].value_counts()

998.00000000    506260
0.00000000      288984
999.00000000    139245
1.00000000      130852
-1.00000000      77623
2.00000000       48169
4.00000000       24727
8.00000000       21928
6.00000000       18264
5.00000000       14479
3.00000000       13989
7.00000000       12585
9.00000000        8479
Name: cluster_label, dtype: int64

In [117]:
pd.crosstab(DS_DF['cluster_label'],DS_DF['acct_spend_decile_1'])

acct_spend_decile_1,-2,-1,1,2,3,4,5,6,7,8,9,10
cluster_label,,,,,,,,,,,,
-1.00000000,0,76294,1171,95,41,11,8,2,0,0,1,0
0.00000000,0,238,41672,32581,31402,31214,32068,32236,30107,25728,19287,12451
1.00000000,0,3,1266,2941,4928,6443,8876,12525,18321,24019,27935,23595
2.00000000,0,14,3969,6534,6757,6635,5996,5110,3693,2752,2303,4406
3.00000000,0,0,225,315,465,629,897,1331,1756,2293,3068,3010
4.00000000,0,0,421,630,774,1054,1470,2156,3136,4099,5392,5595
5.00000000,0,1,196,424,601,782,950,1222,1574,1862,2355,4512
6.00000000,0,1,239,456,632,836,1178,1546,2141,2886,3653,4696
7.00000000,0,1,355,388,420,492,596,726,983,1268,1884,5472


In [112]:
pd.crosstab(DS_DF['acct_spend_decile'],DS_DF['all_zero_3year'])

all_zero_3year,0.0,1.0
acct_spend_decile,,
-1,283,86633
1,44275,1039
2,57328,183
3,64590,66
4,69113,24
5,71794,12
6,74497,3
7,77588,1
8,81461,0


In [115]:
DS_DF[(DS_DF['all_zero_3year']==1)&(DS_DF['acct_spend_decile']!=-1)].head()

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag,all_MR_flag,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,B2B_flag,all_flag,acct_roc_decile,Churned_month,Churned_year,churne_month_year,acct_spend_decile_1,Adv_Publ,Air_Dom,Air_Intl,Air_Net,ATO,Car_Rental,Computer_Software,Consumables,Costco,Courier_Freight,Cruise,Dining,DJ_Prop_BB,DJ_GPC_BB,Education,Entertainment,Everything_Else,Financial_Services,Gas,Healthcare,Insurance,Lodging,Other_Air,Other_Gov,Other_Travel,Outbound,Personal_Services,QF_Air,Retail_DepStores,SBS_B2B,Supermarkets,Coles,Woolworths,Taxi_Limo,Telecomms,Tolls_Private,Transportation,Travel_Agents,Utilities,VA_Air,total,Adv_Publ_ratio,Air_Dom_ratio,Air_Intl_ratio,Air_Net_ratio,ATO_ratio,Car_Rental_ratio,Computer_Software_ratio,Consumables_ratio,Costco_ratio,Courier_Freight_ratio,Cruise_ratio,Dining_ratio,DJ_Prop_BB_ratio,DJ_GPC_BB_ratio,Education_ratio,Entertainment_ratio,Everything_Else_ratio,Financial_Services_ratio,Gas_ratio,Healthcare_ratio,Insurance_ratio,Lodging_ratio,Other_Air_ratio,Other_Gov_ratio,Other_Travel_ratio,Outbound_ratio,Personal_Services_ratio,QF_Air_ratio,Retail_DepStores_ratio,SBS_B2B_ratio,Supermarkets_ratio,Coles_ratio,Woolworths_ratio,Taxi_Limo_ratio,Telecomms_ratio,Tolls_Private_ratio,Transportation_ratio,Travel_Agents_ratio,Utilities_ratio,VA_Air_ratio,count,supp_status,cluster_label,all_zero_3year
649,LAV000000000000650,124.64864865,0.02702703,0.00000000,0.00000000,0.00000000,9.00000000,1.00000000,0.00000000,0,0.00000000,0,35 - 45,M,$0 - $65k,1996-08-01,2100-01-01,3141,Full Fee,NaN,NaN,GREEN,NaN,0,8,1996,1996-08-01,255,8. >20,0,2,4,2,3.33333333,0.10000000,0.03333333,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"21,632.00000000",0,1,1,0.03000000,0.33333333,-1,-1,-1,1,1,1,3.00000000,5.00000000,2.00000000,4,6,3,nan,nan,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,nan,nan,nan,nan,nan,nan,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2100,210001,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,

In [121]:
pd.options.display.float_format = '{:,.2f}'.format
DS_DF[(DS_DF['Churned']==0)&(DS_DF['PIB_Yes']==1)].groupby('acct_spend_decile').agg({'LAV_ID1':'count','
                                                                                     

LAV_ID1 Latest_PIB                             
                    count       mean        std               sum
acct_spend_decile                                                
-1                  51103  66,939.27 254,964.18  3,420,797,462.49
 1                  31582  32,025.62 211,791.65  1,011,433,148.45
 2                  38947  27,399.82 125,680.98  1,067,140,606.50
 3                  41593  26,408.88  80,576.26  1,098,424,440.42
 4                  44589  31,257.13 107,441.48  1,393,723,999.22
 5                  46469  35,039.95 102,544.02  1,628,271,559.07
 6                  48791  42,733.08 120,091.69  2,084,989,656.52
 7                  51610  51,940.23 148,942.09  2,680,635,117.23
 8                  55802  64,626.38 159,904.59  3,606,281,105.24
 9                  60697  89,822.12 180,663.07  5,451,933,431.84
 10                 66659 259,483.30 552,069.95 17,296,897,400.82

In [147]:
df.head()

,LAV_ID1,Gender,IncomeCat,ACQ_TYPE,acq_chn,detailed_fee_type,product_description,days_active,estab_dt,MR_Source_Code,Income,Dob,PostCode,CHANNEL,optout,prod_tier,SuppStatus,AgeCat,myca_app_ind_1,acct_myca_active_1,myca_active_mob_1,acct_spend_lcl Q1,acct_spend_lcl Q2,acct_spend_lcl Q3,acct_spend_lcl Q4,acct_supp_spend_lcl Q1,acct_supp_spend_lcl Q2,acct_supp_spend_lcl Q3,acct_supp_spend_lcl Q4,roc_cnt Q1,roc_cnt Q2,roc_cnt Q3,roc_cnt Q4,roc_supp_cnt Q1,roc_supp_cnt Q2,roc_supp_cnt Q3,roc_supp_cnt Q4,Mean_apps_logs_month,BtStatus_1,Mean_act_supp_month,curr_bal_amt Q1,curr_bal_amt Q2,curr_bal_amt Q3,curr_bal_amt Q4,LOC Q1,LOC Q2,LOC Q3,LOC Q4,Mean_myca_logs_month,revolving_ind,total_apps_count_bin,days_active_bin,active_supps_log_bin,Retail_spend_Q1,Retail_spend_Q2,Retail_spend_Q3,Retail_spend_Q4,Travel_spend_Q1,Travel_spend_Q2,Travel_spend_Q3,Travel_spend_Q4,Util_spend_Q1,Util_spend_Q2,Util_spend_Q3,Util_spend_Q4,PrSr_spend_Q1,PrSr_spend_Q2,PrSr_spend_Q3,PrSr_spend_Q4,Ent_spend_Q1,Ent_spend_Q2,Ent_spend_Q3,Ent_spend_Q4,B2B_spend_Q1,B2B_spend_Q2,B2B_spend_Q3,B2B_spend_Q4,Retail_Cnt_Q1,Retail_Cnt_Q2,Retail_Cnt_Q3,Retail_Cnt_Q4,Travel_Cnt_Q1,Travel_Cnt_Q2,Travel_Cnt_Q3,Travel_Cnt_Q4,Util_Cnt_Q1,Util_Cnt_Q2,Util_Cnt_Q3,Util_Cnt_Q4,PrSr_Cnt_Q1,PrSr_Cnt_Q2,PrSr_Cnt_Q3,PrSr_Cnt_Q4,Ent_Cnt_Q1,Ent_Cnt_Q2,Ent_Cnt_Q3,Ent_Cnt_Q4,B2B_Cnt_Q1,B2B_Cnt_Q2,B2B_Cnt_Q3,B2B_Cnt_Q4,Retail_spend_Q1_ratio,Retail_spend_Q2_ratio,Retail_spend_Q3_ratio,Retail_spend_Q4_ratio,Travel_spend_Q1_ratio,Travel_spend_Q2_ratio,Travel_spend_Q3_ratio,Travel_spend_Q4_ratio,Util_spend_Q1_ratio,Util_spend_Q2_ratio,Util_spend_Q3_ratio,Util_spend_Q4_ratio,PrSr_spend_Q1_ratio,PrSr_spend_Q2_ratio,PrSr_spend_Q3_ratio,PrSr_spend_Q4_ratio,Ent_spend_Q1_ratio,Ent_spend_Q2_ratio,Ent_spend_Q3_ratio,Ent_spend_Q4_ratio,B2B_spend_Q1_ratio,B2B_spend_Q2_ratio,B2B_spend_Q3_ratio,B2B_spend_Q4_ratio,IS_YES,Gift_pnt_Q1,Gift_pnt_Q2,Gift_pnt_Q3,Gift_pnt_Q4,PWP_pnt_Q1,PWP_pnt_Q2,PWP_pnt_Q3,PWP_pnt_Q4,PFC_pnt_Q1,PFC_pnt_Q2,PFC_pnt_Q3,PFC_pnt_Q4,VFF_pnt_Q1,VFF_pnt_Q2,VFF_pnt_Q3,VFF_pnt_Q4,SPWP_pnt_Q1,SPWP_pnt_Q2,SPWP_pnt_Q3,SPWP_pnt_Q4,Air_Oth_pnt_Q1,Air_Oth_pnt_Q2,Air_Oth_pnt_Q3,Air_Oth_pnt_Q4,Travel_pnt_Q1,Travel_pnt_Q2,Travel_pnt_Q3,Travel_pnt_Q4,Others_pnt_Q1,Others_pnt_Q2,Others_pnt_Q3,Others_pnt_Q4,PWP_cnt_Q1,PWP_cnt_Q2,PWP_cnt_Q3,PWP_cnt_Q4,Gift_cnt_Q1,Gift_cnt_Q2,Gift_cnt_Q3,Gift_cnt_Q4,PFC_cnt_Q1,PFC_cnt_Q2,PFC_cnt_Q3,PFC_cnt_Q4,VFF_cnt_Q1,VFF_cnt_Q2,VFF_cnt_Q3,VFF_cnt_Q4,SPWP_cnt_Q1,SPWP_cnt_Q2,SPWP_cnt_Q3,SPWP_cnt_Q4,Air_Oth_cnt_Q1,Air_Oth_cnt_Q2,Air_Oth_cnt_Q3,Air_Oth_cnt_Q4,Travel_cnt_Q1,Travel_cnt_Q2,Travel_cnt_Q3,Travel_cnt_Q4,Others_cnt_Q1,Others_cnt_Q2,Others_cnt_Q3,Others_cnt_Q4,PWP_RdPB_Q1,PWP_RdPB_Q2,PWP_RdPB_Q3,PWP_RdPB_Q4,Gift_RdPB_Q1,Gift_RdPB_Q2,Gift_RdPB_Q3,Gift_RdPB_Q4,PFC_RdPB_Q1,PFC_RdPB_Q2,PFC_RdPB_Q3,PFC_RdPB_Q4,VFF_RdPB_Q1,VFF_RdPB_Q2,VFF_RdPB_Q3,VFF_RdPB_Q4,SPWP_RdPB_Q1,SPWP_RdPB_Q2,SPWP_RdPB_Q3,SPWP_RdPB_Q4,Air_Oth_RdPB_Q1,Air_Oth_RdPB_Q2,Air_Oth_RdPB_Q3,Air_Oth_RdPB_Q4,Travel_RdPB_Q1,Travel_RdPB_Q2,Travel_RdPB_Q3,Travel_RdPB_Q4,Others_RdPB_Q1,Others_RdPB_Q2,Others_RdPB_Q3,Others_RdPB_Q4,RDM_pnt_Q1,RDM_pnt_Q2,RDM_pnt_Q3,RDM_pnt_Q4,RDM_cnt_Q1,RDM_cnt_Q2,RDM_cnt_Q3,RDM_cnt_Q4,All_RdPB_Q1,All_RdPB_Q2,All_RdPB_Q3,All_RdPB_Q4,MR_YES,total_sent Q1,total_sent Q2,total_sent Q3,total_sent Q4,total_opens Q1,total_opens Q2,total_opens Q3,total_opens Q4,total_clicks Q1,total_clicks Q2,total_clicks Q3,total_clicks Q4,E_O/S_Q1,E_O/S_Q2,E_O/S_Q3,E_O/S_Q4,E_C/O_Q1,E_C/O_Q2,E_C/O_Q3,E_C/O_Q4,EMAIL_YES,PIB_Q1,PIB_Q2,PIB_Q3,PIB_Q4,PIB_YES,Phone_pay_Q1,Phone_pay_Q2,Phone_pay_Q3,Phone_pay_Q4,Watch_pay_Q1,Watch_pay_Q2,Watch_pay_Q3,Watch_pay_Q4,Other_pay_Q1,Other_pay_Q2,Other_pay_Q3,Other_pay_Q4,MP_yes,FF_taken,FF_Count,FH_taken,FH_Count,MGMee_count,BONUS_POINT_CT_MGMee,MGMer_count,BONUS_POINT_CT_MGMer,cluster_label,all_zero_3year,ENROL_DATE,CM_ACCT_STAT_CD,MBR_STAT_CD,MENR_YES,# of Local Champions,LC_YES,cancel_dt,months_active,estab_dt_1,pred_prob,pred_value_0.5,pred_value_0.75,pred_

In [148]:
df.shape

(811094, 304)

In [149]:
DS_DF.head()

,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag,all_MR_flag,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,B2B_flag,all_flag,acct_roc_decile,Churned_month,Churned_year,churne_month_year,acct_spend_decile_1,Adv_Publ,Air_Dom,Air_Intl,Air_Net,ATO,Car_Rental,Computer_Software,Consumables,Costco,Courier_Freight,Cruise,Dining,DJ_Prop_BB,DJ_GPC_BB,Education,Entertainment,Everything_Else,Financial_Services,Gas,Healthcare,Insurance,Lodging,Other_Air,Other_Gov,Other_Travel,Outbound,Personal_Services,QF_Air,Retail_DepStores,SBS_B2B,Supermarkets,Coles,Woolworths,Taxi_Limo,Telecomms,Tolls_Private,Transportation,Travel_Agents,Utilities,VA_Air,total,Adv_Publ_ratio,Air_Dom_ratio,Air_Intl_ratio,Air_Net_ratio,ATO_ratio,Car_Rental_ratio,Computer_Software_ratio,Consumables_ratio,Costco_ratio,Courier_Freight_ratio,Cruise_ratio,Dining_ratio,DJ_Prop_BB_ratio,DJ_GPC_BB_ratio,Education_ratio,Entertainment_ratio,Everything_Else_ratio,Financial_Services_ratio,Gas_ratio,Healthcare_ratio,Insurance_ratio,Lodging_ratio,Other_Air_ratio,Other_Gov_ratio,Other_Travel_ratio,Outbound_ratio,Personal_Services_ratio,QF_Air_ratio,Retail_DepStores_ratio,SBS_B2B_ratio,Supermarkets_ratio,Coles_ratio,Woolworths_ratio,Taxi_Limo_ratio,Telecomms_ratio,Tolls_Private_ratio,Transportation_ratio,Travel_Agents_ratio,Utilities_ratio,VA_Air_ratio,count,supp_status,cluster_label,all_zero_3year
0,LAV000000000000001,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"33,468.50",0,0,1,nan,nan,-1,-1,-1,-1,-1,-1,-1.00,-1.00,-1.00,4,7,3,9.00,"2,014.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,nan,nan,nan,nan,nan,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,9,2014,201409,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0,0,998.00,nan
1,LAV000000000000002,119.65,1.32,0.00,0.00,0.00,37.00,1.00,0.00,

In [150]:
DS_DF.shape

(1305584, 231)

In [151]:
DS_DF_1=DS_DF[['LAV_ID1','acct_spend_decile','acct_spend_decile_1','all_MR_flag','count','supp_status']]

In [152]:
DS_DF_1.shape

(1305584, 6)

In [156]:
df1=df.merge(DS_DF_1,on='LAV_ID1',how='inner')

In [157]:
df1.shape

(811094, 309)

In [160]:
df1.head()

,LAV_ID1,Gender,IncomeCat,ACQ_TYPE,acq_chn,detailed_fee_type,product_description,days_active,estab_dt,MR_Source_Code,Income,Dob,PostCode,CHANNEL,optout,prod_tier,SuppStatus,AgeCat,myca_app_ind_1,acct_myca_active_1,myca_active_mob_1,acct_spend_lcl Q1,acct_spend_lcl Q2,acct_spend_lcl Q3,acct_spend_lcl Q4,acct_supp_spend_lcl Q1,acct_supp_spend_lcl Q2,acct_supp_spend_lcl Q3,acct_supp_spend_lcl Q4,roc_cnt Q1,roc_cnt Q2,roc_cnt Q3,roc_cnt Q4,roc_supp_cnt Q1,roc_supp_cnt Q2,roc_supp_cnt Q3,roc_supp_cnt Q4,Mean_apps_logs_month,BtStatus_1,Mean_act_supp_month,curr_bal_amt Q1,curr_bal_amt Q2,curr_bal_amt Q3,curr_bal_amt Q4,LOC Q1,LOC Q2,LOC Q3,LOC Q4,Mean_myca_logs_month,revolving_ind,total_apps_count_bin,days_active_bin,active_supps_log_bin,Retail_spend_Q1,Retail_spend_Q2,Retail_spend_Q3,Retail_spend_Q4,Travel_spend_Q1,Travel_spend_Q2,Travel_spend_Q3,Travel_spend_Q4,Util_spend_Q1,Util_spend_Q2,Util_spend_Q3,Util_spend_Q4,PrSr_spend_Q1,PrSr_spend_Q2,PrSr_spend_Q3,PrSr_spend_Q4,Ent_spend_Q1,Ent_spend_Q2,Ent_spend_Q3,Ent_spend_Q4,B2B_spend_Q1,B2B_spend_Q2,B2B_spend_Q3,B2B_spend_Q4,Retail_Cnt_Q1,Retail_Cnt_Q2,Retail_Cnt_Q3,Retail_Cnt_Q4,Travel_Cnt_Q1,Travel_Cnt_Q2,Travel_Cnt_Q3,Travel_Cnt_Q4,Util_Cnt_Q1,Util_Cnt_Q2,Util_Cnt_Q3,Util_Cnt_Q4,PrSr_Cnt_Q1,PrSr_Cnt_Q2,PrSr_Cnt_Q3,PrSr_Cnt_Q4,Ent_Cnt_Q1,Ent_Cnt_Q2,Ent_Cnt_Q3,Ent_Cnt_Q4,B2B_Cnt_Q1,B2B_Cnt_Q2,B2B_Cnt_Q3,B2B_Cnt_Q4,Retail_spend_Q1_ratio,Retail_spend_Q2_ratio,Retail_spend_Q3_ratio,Retail_spend_Q4_ratio,Travel_spend_Q1_ratio,Travel_spend_Q2_ratio,Travel_spend_Q3_ratio,Travel_spend_Q4_ratio,Util_spend_Q1_ratio,Util_spend_Q2_ratio,Util_spend_Q3_ratio,Util_spend_Q4_ratio,PrSr_spend_Q1_ratio,PrSr_spend_Q2_ratio,PrSr_spend_Q3_ratio,PrSr_spend_Q4_ratio,Ent_spend_Q1_ratio,Ent_spend_Q2_ratio,Ent_spend_Q3_ratio,Ent_spend_Q4_ratio,B2B_spend_Q1_ratio,B2B_spend_Q2_ratio,B2B_spend_Q3_ratio,B2B_spend_Q4_ratio,IS_YES,Gift_pnt_Q1,Gift_pnt_Q2,Gift_pnt_Q3,Gift_pnt_Q4,PWP_pnt_Q1,PWP_pnt_Q2,PWP_pnt_Q3,PWP_pnt_Q4,PFC_pnt_Q1,PFC_pnt_Q2,PFC_pnt_Q3,PFC_pnt_Q4,VFF_pnt_Q1,VFF_pnt_Q2,VFF_pnt_Q3,VFF_pnt_Q4,SPWP_pnt_Q1,SPWP_pnt_Q2,SPWP_pnt_Q3,SPWP_pnt_Q4,Air_Oth_pnt_Q1,Air_Oth_pnt_Q2,Air_Oth_pnt_Q3,Air_Oth_pnt_Q4,Travel_pnt_Q1,Travel_pnt_Q2,Travel_pnt_Q3,Travel_pnt_Q4,Others_pnt_Q1,Others_pnt_Q2,Others_pnt_Q3,Others_pnt_Q4,PWP_cnt_Q1,PWP_cnt_Q2,PWP_cnt_Q3,PWP_cnt_Q4,Gift_cnt_Q1,Gift_cnt_Q2,Gift_cnt_Q3,Gift_cnt_Q4,PFC_cnt_Q1,PFC_cnt_Q2,PFC_cnt_Q3,PFC_cnt_Q4,VFF_cnt_Q1,VFF_cnt_Q2,VFF_cnt_Q3,VFF_cnt_Q4,SPWP_cnt_Q1,SPWP_cnt_Q2,SPWP_cnt_Q3,SPWP_cnt_Q4,Air_Oth_cnt_Q1,Air_Oth_cnt_Q2,Air_Oth_cnt_Q3,Air_Oth_cnt_Q4,Travel_cnt_Q1,Travel_cnt_Q2,Travel_cnt_Q3,Travel_cnt_Q4,Others_cnt_Q1,Others_cnt_Q2,Others_cnt_Q3,Others_cnt_Q4,PWP_RdPB_Q1,PWP_RdPB_Q2,PWP_RdPB_Q3,PWP_RdPB_Q4,Gift_RdPB_Q1,Gift_RdPB_Q2,Gift_RdPB_Q3,Gift_RdPB_Q4,PFC_RdPB_Q1,PFC_RdPB_Q2,PFC_RdPB_Q3,PFC_RdPB_Q4,VFF_RdPB_Q1,VFF_RdPB_Q2,VFF_RdPB_Q3,VFF_RdPB_Q4,SPWP_RdPB_Q1,SPWP_RdPB_Q2,SPWP_RdPB_Q3,SPWP_RdPB_Q4,Air_Oth_RdPB_Q1,Air_Oth_RdPB_Q2,Air_Oth_RdPB_Q3,Air_Oth_RdPB_Q4,Travel_RdPB_Q1,Travel_RdPB_Q2,Travel_RdPB_Q3,Travel_RdPB_Q4,Others_RdPB_Q1,Others_RdPB_Q2,Others_RdPB_Q3,Others_RdPB_Q4,RDM_pnt_Q1,RDM_pnt_Q2,RDM_pnt_Q3,RDM_pnt_Q4,RDM_cnt_Q1,RDM_cnt_Q2,RDM_cnt_Q3,RDM_cnt_Q4,All_RdPB_Q1,All_RdPB_Q2,All_RdPB_Q3,All_RdPB_Q4,MR_YES,total_sent Q1,total_sent Q2,total_sent Q3,total_sent Q4,total_opens Q1,total_opens Q2,total_opens Q3,total_opens Q4,total_clicks Q1,total_clicks Q2,total_clicks Q3,total_clicks Q4,E_O/S_Q1,E_O/S_Q2,E_O/S_Q3,E_O/S_Q4,E_C/O_Q1,E_C/O_Q2,E_C/O_Q3,E_C/O_Q4,EMAIL_YES,PIB_Q1,PIB_Q2,PIB_Q3,PIB_Q4,PIB_YES,Phone_pay_Q1,Phone_pay_Q2,Phone_pay_Q3,Phone_pay_Q4,Watch_pay_Q1,Watch_pay_Q2,Watch_pay_Q3,Watch_pay_Q4,Other_pay_Q1,Other_pay_Q2,Other_pay_Q3,Other_pay_Q4,MP_yes,FF_taken,FF_Count,FH_taken,FH_Count,MGMee_count,BONUS_POINT_CT_MGMee,MGMer_count,BONUS_POINT_CT_MGMer,cluster_label,all_zero_3year,ENROL_DATE,CM_ACCT_STAT_CD,MBR_STAT_CD,MENR_YES,# of Local Champions,LC_YES,cancel_dt,months_active,estab_dt_1,pred_prob,pred_value_0.5,pred_value_0.75,pred_

In [159]:
df1.to_csv("/Users/sri.t/Desktop/AMEX1/Churn_Data/cluster_data_with_churn.csv",header=True,sep=',',encoding='utf-8')

In [161]:
DS_DF.to_csv('/Users/sri.t/Desktop/amex_churners_estimates_4.csv',header=True,sep=',',encoding='utf-8')

In [31]:
for i in dir():
    if type(globals()[i]) == pd.DataFrame:
        print(i)

DS_DF


In [11]:
pd.crosstab(DS_DF[DS_DF['Churned']==0]['acct_spend_decile'],DS_DF[DS_DF['Churned']==0]['supp_status']).apply(lambda r: r/r.sum(), axis=1)

supp_status,0,1
acct_spend_decile,,
-1,0.999839,0.000161
1,0.924791,0.075209
2,0.898263,0.101737
3,0.875634,0.124366
4,0.845134,0.154866
5,0.806841,0.193159
6,0.768242,0.231758
7,0.717009,0.282991
8,0.643744,0.356256


In [109]:
DS_DF.head()

,Unnamed: 0,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag,all_MR_flag,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,B2B_flag,all_flag,acct_roc_decile,Churned_month,Churned_year,churne_month_year,acct_spend_decile_1,Adv_Publ,Air_Dom,Air_Intl,Air_Net,ATO,Car_Rental,Computer_Software,Consumables,Costco,Courier_Freight,Cruise,Dining,DJ_Prop_BB,DJ_GPC_BB,Education,Entertainment,Everything_Else,Financial_Services,Gas,Healthcare,Insurance,Lodging,Other_Air,Other_Gov,Other_Travel,Outbound,Personal_Services,QF_Air,Retail_DepStores,SBS_B2B,Supermarkets,Coles,Woolworths,Taxi_Limo,Telecomms,Tolls_Private,Transportation,Travel_Agents,Utilities,VA_Air,total,Adv_Publ_ratio,Air_Dom_ratio,Air_Intl_ratio,Air_Net_ratio,ATO_ratio,Car_Rental_ratio,Computer_Software_ratio,Consumables_ratio,Costco_ratio,Courier_Freight_ratio,Cruise_ratio,Dining_ratio,DJ_Prop_BB_ratio,DJ_GPC_BB_ratio,Education_ratio,Entertainment_ratio,Everything_Else_ratio,Financial_Services_ratio,Gas_ratio,Healthcare_ratio,Insurance_ratio,Lodging_ratio,Other_Air_ratio,Other_Gov_ratio,Other_Travel_ratio,Outbound_ratio,Personal_Services_ratio,QF_Air_ratio,Retail_DepStores_ratio,SBS_B2B_ratio,Supermarkets_ratio,Coles_ratio,Woolworths_ratio,Taxi_Limo_ratio,Telecomms_ratio,Tolls_Private_ratio,Transportation_ratio,Travel_Agents_ratio,Utilities_ratio,VA_Air_ratio,count,supp_status,cluster_label,all_zero_3year
0,0,LAV000000000000001,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33468.500000,0,0,1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,4,7,3,9.0,2014.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,9,2014,201409,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,998.0,NaN
1,1,LA

In [102]:
DS_DF['product_description'].unique()

array(['GREEN', 'GOLD', 'PLATINUM', 'REWARDS ADVANTAGE', 'CENTURION',
       'AFFINITY DIST', 'AFFINITY PLATINUM', 'AFFINITY GOLD',
       'AFFINITY GOLD PREMIUM', 'PLATINUM REWARDS', 'PLATINUM EDGE',
       'AFFINITY BLUE', 'AFFINITY GENERIC', 'AFFINITY GOLD STANDARD',
       'AFFINITY PLATINUM EDGE', 'QANTAS DISCOVERY 1FOR1', 'CARE', 'BLUE',
       'QANTAS DISCOVERY', 'QANTAS CLASSIC', 'VELOCITY ESCAPE',
       'GOLD ASCENT - NO FEE', 'QANTAS PREMIUM', 'GOLD ASCENT - FEE',
       'QANTAS ULTIMATE', 'VELOCITY GOLD', 'PLATINUM CASHBACK',
       'ESSENTIAL', 'EXPLORER', 'DEBT LIQUIDATION', 'VELOCITY PLATINUM',
       'PLATINUM RWRDS MAXIMIZER', 'LOW RATE', 'BLUE SKY',
       'PLATINUM EDGE TC', 'REWARDS MAXIMIZER', 'GOLD CHOICES',
       'GOLD CASHBACK', 'DAVID JONES GPC', 'DAVID JONES PLATINUM',
       'DAVID JONES STORECARD TRANSFER', 'AFFINITY ESSENTIAL',
       'WESTPAC PLATINUMBB', 'WESTPAC PLATINUM', 'WESTPAC BLACK'],
      dtype=object)

In [34]:
mr_c=['LAV_ID1','PWP_flag', 'VFF_flag', 'AirForeign_flag','GiftCard_flag','PFC_flag','S&PWP_flag','Other_flag','Helloworld_flag',
       'FX_flag','Webjet_flag','Hotel_flag','Airlocal_flag','Ent_flag','AirLocal_flag','AmexTravel_flag','RewardsCon_flag','QFF_flag','Donations_flag','Safekey_flag']

In [16]:
#pd.crosstab(DS_DF[DS_DF['Churned']==0]['optout'],DS_DF[DS_DF['Churned']==0]['MR_Yes'])

In [21]:
DS_DF.groupby('acct_spend_decile')[mr_c].agg('sum').apply(lambda r: r/r.sum(), axis=1)

,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag
acct_spend_decile,,,,,,,,,,,,,,,,,,,
-1,0.012543,0.093925,0.029753,0.482243,0.065558,0.021731,0.091009,0.007584,0.003719,0.004521,0.005396,0.002261,0.125210,0.004521,0.019106,0.000292,0.028513,0.002115,0.000000
1,0.019950,0.032449,0.011222,0.240678,0.300202,0.011459,0.112219,0.001158,0.001722,0.001247,0.005017,0.000327,0.209743,0.001098,0.006888,0.000059,0.043789,0.000772,0.000000
2,0.023857,0.038673,0.013077,0.170238,0.192098,0.032918,0.094761,0.001245,0.001546,0.001224,0.003951,0.000429,0.375564,0.001245,0.009491,0.000021,0.039124,0.000537,0.000000
3,0.027130,0.042070,0.014035,0.117229,0.127576,0.053851,0.073640,0.001520,0.001281,0.001485,0.003773,0.000410,0.490242,0.001076,0.015452,0.000017,0.028855,0.000341,0.000017
4,0.030082,0.041965,0.014842,0.098212,0.109256,0.063183,0.068116,0.001958,0.001325,0.001737,0.003931,0.000633,0.514828,0.001222,0.017949,0.000000,0.030347,0.000412,0.000000
5,0.032739,0.047810,0.016269,0.086174,0.097512,0.064066,0.063751,0.001993,0.001866,0.002396,0.003771,0.000731,0.524321,0.001438,0.022385,0.000050,0.032449,0.000227,0.000050
6,0.031571,0.051578,0.018069,0.080970,0.083357,0.066033,0.061817,0.002836,0.002201,0.003187,0.003712,0.000778,0.529961,0.001675,0.027355,0.000044,0.034725,0.000120,0.000011
7,0.029211,0.050404,0.020505,0.076289,0.072746,0.066451,0.061881,0.003929,0.002855,0.004108,0.004042,0.000914,0.532618,0.001819,0.030955,0.000028,0.041075,0.000122,0.000047
8,0.027305,0.051565,0.022484,0.071610,0.061878,0.063219,0.056815,0.006009,0.003893,0.005807,0.003893,0.001114,0.538878,0.002390,0.035413,0.000032,0.047527,0.000145,0.000024


In [103]:
DS_DF[DS_DF['product_description']!='CENTURION'].groupby('acct_spend_decile')[mr_c].agg('sum').apply(lambda r: r/r.sum(), axis=1)

,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag
acct_spend_decile,,,,,,,,,,,,,,,,,,,
-1,0.012590,0.093910,0.029644,0.484043,0.065803,0.021666,0.091348,0.007612,0.003733,0.004392,0.005270,0.002269,0.124286,0.004538,0.018519,0.000293,0.027961,0.002123,0.000000
1,0.019951,0.032451,0.011223,0.240692,0.300220,0.011460,0.112226,0.001158,0.001722,0.001247,0.005018,0.000327,0.209697,0.001099,0.006888,0.000059,0.043792,0.000772,0.000000
2,0.023860,0.038636,0.013036,0.170264,0.192127,0.032923,0.094775,0.001246,0.001546,0.001224,0.003952,0.000430,0.375599,0.001246,0.009493,0.000021,0.039087,0.000537,0.000000
3,0.027133,0.042074,0.014036,0.117222,0.127587,0.053856,0.073646,0.001520,0.001281,0.001486,0.003774,0.000410,0.490267,0.001076,0.015436,0.000017,0.028823,0.000342,0.000017
4,0.030078,0.041950,0.014818,0.098216,0.109278,0.063190,0.068139,0.001959,0.001326,0.001738,0.003933,0.000633,0.514884,0.001223,0.017941,0.000000,0.030284,0.000412,0.000000
5,0.032745,0.047818,0.016271,0.086188,0.097515,0.064077,0.063749,0.001993,0.001867,0.002397,0.003771,0.000732,0.524331,0.001438,0.022376,0.000050,0.032404,0.000227,0.000050
6,0.031578,0.051567,0.018040,0.080988,0.083375,0.066036,0.061830,0.002837,0.002202,0.003187,0.003713,0.000778,0.530001,0.001676,0.027350,0.000044,0.034667,0.000120,0.000011
7,0.029227,0.050394,0.020431,0.076321,0.072776,0.066469,0.061905,0.003913,0.002847,0.004111,0.004035,0.000905,0.532730,0.001820,0.030943,0.000028,0.040975,0.000123,0.000047
8,0.027320,0.051544,0.022430,0.071662,0.061914,0.063240,0.056855,0.006014,0.003888,0.005812,0.003888,0.001115,0.539036,0.002393,0.035379,0.000032,0.047309,0.000145,0.000024


In [105]:
DS_DF[~DS_DF['product_description'].isin(['CENTURION','DAVID JONES GPC', 'DAVID JONES PLATINUM'])].groupby('acct_spend_decile')[mr_c].agg('sum').apply(lambda r: r/r.sum(), axis=1)

,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag
acct_spend_decile,,,,,,,,,,,,,,,,,,,
-1,0.012387,0.116837,0.037459,0.409176,0.053315,0.029333,0.099296,0.010108,0.004459,0.005946,0.005450,0.002874,0.141710,0.005847,0.024775,0.000396,0.037756,0.002874,0.000000
1,0.012491,0.067516,0.019411,0.246350,0.043970,0.032577,0.066841,0.003123,0.003460,0.003038,0.003460,0.000591,0.356486,0.002025,0.018314,0.000169,0.118913,0.001266,0.000000
2,0.006811,0.055440,0.019352,0.119684,0.033347,0.063663,0.048214,0.002243,0.002076,0.001993,0.004693,0.000706,0.548630,0.001869,0.017733,0.000042,0.072799,0.000706,0.000000
3,0.004865,0.052762,0.017740,0.068136,0.027631,0.084774,0.038785,0.002204,0.001398,0.002096,0.004408,0.000591,0.625695,0.001317,0.023706,0.000027,0.043516,0.000323,0.000027
4,0.005560,0.050999,0.018736,0.057150,0.028148,0.093900,0.040208,0.002627,0.001313,0.002211,0.004312,0.000832,0.622803,0.001270,0.025762,0.000000,0.043842,0.000328,0.000000
5,0.007095,0.057426,0.020075,0.052516,0.028596,0.091708,0.038904,0.002690,0.002022,0.002997,0.004170,0.000848,0.612821,0.001625,0.030979,0.000072,0.045168,0.000217,0.000072
6,0.007234,0.061127,0.021731,0.050411,0.028695,0.090482,0.040071,0.003617,0.002356,0.003857,0.004187,0.000900,0.599967,0.002116,0.036574,0.000060,0.046479,0.000120,0.000015
7,0.009631,0.058939,0.024152,0.049852,0.028559,0.087275,0.041917,0.004741,0.002823,0.005113,0.004395,0.001077,0.587059,0.002129,0.039243,0.000037,0.052897,0.000099,0.000062
8,0.010912,0.059616,0.025884,0.049893,0.029823,0.078830,0.041007,0.007204,0.003748,0.006892,0.004191,0.001330,0.576764,0.002781,0.042599,0.000040,0.058346,0.000111,0.000030


In [22]:
mr_c1=['PWP_pnt','VFF_pnt','Air - Foreign_pnt','Gift Card_pnt','Points for credit_pnt','S&PWP_pnt','Other_pnt','Helloworld_pnt',
       'FX_pnt','Webjet_pnt','Hotels_pnt','Air - local_pnt','Entertainment_pnt','Air - Local_pnt','Amex Travel Online_pnt','Rewards Conceirge_pnt','QFF_pnt','Donations_pnt','Safekey_pnt']

In [27]:
DS_DF.groupby('acct_spend_decile')[mr_c1].agg('mean')

,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt
acct_spend_decile,,,,,,,,,,,,,,,,,,,
-1,39451.122578,48165.844525,84605.201552,20314.778864,30811.002966,46419.578167,28980.676397,161855.931662,59440.352941,88348.008065,38972.144939,72431.120430,25612.500000,66052.720430,99940.307502,3.098852e+05,65274.686323,7965.517241,NaN
1,23183.131349,35127.570147,51918.117108,16550.943867,22833.235265,16252.316984,20940.706484,201908.508547,65014.568966,58212.690476,38972.682446,57027.000000,19255.566667,36855.342342,46641.618370,1.135284e+06,7890.397104,5615.384615,NaN
2,20101.354100,29278.513504,61201.346137,15231.429427,24126.928036,14772.299050,17236.364076,117968.804598,51633.020833,48822.766082,45442.572464,66841.675000,17824.861953,31821.126437,44312.502177,6.389500e+04,10702.353659,5160.000000,NaN
3,17612.347761,30918.880730,63397.945603,15714.522087,24875.346479,16544.110931,17959.526625,124925.059925,56870.786667,53391.868774,52218.898944,59898.718750,14638.226115,41569.410053,44888.333228,2.632560e+05,9573.479545,12075.000000,6273.000000
4,17915.104316,29849.498659,58675.621817,17369.800197,24545.441094,15680.914624,18181.588305,103579.493734,42222.941667,52463.598366,48933.978955,64906.748062,15955.554656,39219.723896,45783.580876,NaN,10858.639780,4071.428571,NaN
5,18418.933150,28927.918779,56115.172069,19289.235211,25776.271621,16707.466420,18977.694536,132063.213608,42470.452703,53314.482895,44421.982512,44735.051437,17020.549005,31376.139819,42982.737587,1.100635e+05,10509.594640,5833.333333,16247.250000
6,21461.516035,32253.324957,58742.624035,21237.071761,28204.636424,19254.754052,21782.606986,125846.831403,46017.788308,48560.910452,55711.253687,58613.279343,18749.340000,42669.878774,41039.026874,1.890725e+05,13938.906041,3818.181818,10676.000000
7,23840.715585,34285.248514,60017.660106,25279.971951,34055.287967,22048.144662,24394.868368,118225.137090,50316.064466,49940.068005,49173.053109,53842.306873,21357.535430,45209.169862,42430.504781,1.792207e+05,14973.706205,6307.692308,17255.000000
8,28544.916843,38073.090255,66032.188916,29065.511587,43900.460861,28838.152616,30332.310661,113792.847241,59258.221619,58593.573551,48478.282312,59239.998447,22871.843575,50396.934913,49310.829936,1.790098e+05,19439.918602,6500.000000,36730.000000


In [106]:
DS_DF[~DS_DF['product_description'].isin(['CENTURION'])].groupby('acct_spend_decile')[mr_c1].agg('mean')

,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt
acct_spend_decile,,,,,,,,,,,,,,,,,,,
-1,39451.122578,47785.416795,84264.299547,20314.778864,30811.002966,44801.068560,28980.676397,161855.931662,59440.352941,89367.275000,34617.204521,72431.120430,24661.212121,66052.720430,85404.904250,3.098852e+05,64241.631304,7965.517241,NaN
1,23183.131349,35127.570147,51918.117108,16550.943867,22833.235265,16252.316984,20940.706484,201908.508547,65014.568966,58212.690476,38972.682446,57027.000000,19255.566667,36855.342342,46641.618370,1.135284e+06,7890.397104,5615.384615,NaN
2,20101.354100,29238.312667,60145.631050,15231.429427,24126.928036,14772.299050,17236.364076,117968.804598,51633.020833,48822.766082,45442.572464,66841.675000,17824.861953,31821.126437,44312.502177,6.389500e+04,10334.509542,5160.000000,NaN
3,17612.347761,30918.880730,63397.945603,15707.707014,24875.346479,16544.110931,17959.526625,124925.059925,56870.786667,53391.868774,52218.898944,59898.718750,14638.226115,41569.410053,44519.233265,2.632560e+05,9029.752625,12075.000000,6273.000000
4,17846.858652,29673.644023,58494.652543,17363.649867,24532.664723,15674.015769,18181.588305,103579.493734,42222.941667,52463.598366,48933.978955,64906.748062,15955.554656,39219.723896,45420.780533,NaN,10209.620884,4071.428571,NaN
5,18418.933150,28927.918779,56115.172069,19289.235211,25729.159511,16707.466420,18976.107218,132063.213608,42470.452703,53314.482895,44421.982512,44735.051437,17020.549005,31376.139819,42894.426415,1.100635e+05,10211.555278,5833.333333,16247.250000
6,21461.516035,32215.291535,58225.853365,21237.071761,28204.636424,19256.661292,21782.606986,125846.831403,46017.788308,48560.910452,55711.253687,58613.279343,18749.340000,42669.878774,40969.183380,1.890725e+05,13827.184101,3818.181818,10676.000000
7,23840.715585,34167.270253,59696.613116,25261.167747,34055.036029,22033.474472,24391.901549,114788.019679,48878.720309,49940.068005,49193.784541,52840.664236,21357.535430,45209.169862,42373.145753,1.792207e+05,14654.806525,6307.692308,17255.000000
8,28395.458535,37977.614943,65292.488148,29065.124209,43667.732522,28697.170179,30332.784404,113792.847241,58895.550562,58593.573551,48554.744437,59239.998447,22871.843575,50396.934913,48771.115227,1.790098e+05,18371.106911,6500.000000,36730.000000


In [107]:
DS_DF[~DS_DF['product_description'].isin(['CENTURION','DAVID JONES GPC', 'DAVID JONES PLATINUM'])].groupby('acct_spend_decile')[mr_c1].agg('mean')

,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt
acct_spend_decile,,,,,,,,,,,,,,,,,,,
-1,46054.059333,50085.870864,88055.511420,23482.268337,39159.014870,44801.068560,31336.291560,164384.498950,62890.888889,89367.275000,32989.795009,74525.887356,24271.687500,68867.096045,85609.179795,3.098852e+05,64409.761570,7965.517241,NaN
1,40068.558108,40330.026464,66971.406957,18834.093710,30434.149072,16252.316984,26814.304938,211494.860360,79229.406504,62541.916667,40215.853659,73181.000000,19645.122222,44458.902778,47378.603511,1.135284e+06,7798.305805,6633.333333,NaN
2,35699.392276,33940.450053,70017.628858,17712.391279,29936.700913,14772.299050,20095.069337,117685.104938,59404.560000,52617.909722,54792.330383,76897.382353,18832.541667,33500.607407,44809.568687,6.389500e+04,10445.814310,5823.529412,NaN
3,34328.041897,34925.564503,72212.361544,17419.041793,34263.089062,16544.110931,21426.519058,132524.626016,66693.307692,56515.994658,61983.922764,60565.420455,14858.631818,46711.119048,44691.306423,2.632560e+05,9173.632275,17625.000000,6273.000000
4,29872.281496,32268.059833,63735.286750,19639.495332,32191.964749,15674.015769,20214.455453,110615.405556,45413.229167,55205.827793,57707.262509,69698.820175,16963.924051,43162.570402,45124.789864,NaN,10266.481387,4200.000000,NaN
5,27090.556446,30532.482694,60560.474493,20947.322478,34734.678527,16707.466420,19923.522815,135544.951342,44308.830357,56436.745482,48170.330487,49516.673404,16422.954846,34068.128956,43201.409690,1.100635e+05,10346.092583,6000.000000,16247.250000
6,30811.756397,33344.094944,62022.111067,22498.318031,36502.704099,19256.661292,23147.382653,129555.756570,46241.534713,49972.062030,59430.609319,62494.419444,19099.639344,41679.804627,41056.862171,1.890725e+05,13998.890384,3375.000000,10676.000000
7,32791.695351,34987.625508,61121.671814,26658.406608,42947.457015,22033.474472,24311.258678,117615.841166,50178.961404,50492.298184,51272.524480,53057.487739,21308.766958,44969.401841,42227.887590,1.792207e+05,14830.984534,6125.000000,17255.000000
8,36012.203124,38050.900316,65760.477541,29726.776313,51508.244007,28697.170179,30191.090281,114559.189297,62341.977653,58953.168689,50005.603658,58657.185245,22718.819704,50820.208837,48454.471202,1.790098e+05,18491.150156,7909.090909,36730.000000


In [50]:
ddd=DS_DF.pivot_table(index=['acct_spend_decile'], columns=['product_description'], values=['acct_spend_lcl'],aggfunc='sum').reset_index().fillna(value=0)

In [54]:
ddd=DS_DF.pivot_table(index=['acct_spend_decile'], columns=['revolving_ind'], values=['acct_spend_lcl'],aggfunc='sum').reset_index().fillna(value=0)

In [63]:
ddd=DS_DF[DS_DF['Churned']==0].pivot_table(index=['acct_spend_decile'], columns=['tenure_bin'], values=['acct_spend_lcl'],aggfunc='sum').reset_index().fillna(value=0)

In [71]:
#df=pd.read_csv("/Users/sri.t/Desktop/AMEX1/Churn_Data/cluster_data_with_churn.csv",sep=',')
df=pd.read_csv("/Users/sri.t/Desktop/MR_prepared_by_Level2.csv",sep=',')
df.shape

(799324, 246)

In [67]:
DS_DF.head()

,Unnamed: 0,LAV_ID1,acct_spend_lcl,roc_cnt,acct_supp_spend_lcl,roc_supp_cnt,myca_active_app,optout,acct_myca_active,myca_app_ind,SuppStatus_1,revolving_ind,BtStatus_1,AgeCat,Gender,IncomeCat,estab_dt,cancel_dt,PostCode,detailed_fee_type,ACQ_TYPE,acq_chn,product_description,prod_tier,Churned,estab_dt_month,estab_dt_year,estab_dt_1,months_active,tenure_bin,acct_spend_zero_flag,acct_spend_quartile,acct_spend_decile,acct_spend_pentile,total_sent,total_opens,total_clicks,rdm_cntsum,rdm_pntsum,PWP_pnt,VFF_pnt,Air - Foreign_pnt,Gift Card_pnt,Points for credit_pnt,S&PWP_pnt,Other_pnt,Helloworld_pnt,FX_pnt,Webjet_pnt,Hotels_pnt,Air - local_pnt,Entertainment_pnt,Air - Local_pnt,Amex Travel Online_pnt,Rewards Conceirge_pnt,QFF_pnt,Donations_pnt,Safekey_pnt,PWP_cnt,VFF_cnt,Air - Foreign_cnt,Gift Card_cnt,Points for credit_cnt,S&PWP_cnt,Other_cnt,Helloworld_cnt,FX_cnt,Webjet_cnt,Hotels_cnt,Air - local_cnt,Entertainment_cnt,Air - Local_cnt,Amex Travel Online_cnt,Rewards Conceirge_cnt,QFF_cnt,Donations_cnt,Safekey_cnt,Latest_PIB,MR_Yes,EM_Yes,PIB_Yes,E_O/S,E_C/O,MR_pnt_pentile,MR_pnt_decile,MR_pnt_quartile,E_O/S_pentile,E_O/S_decile,E_O/S_quartile,E_C/O_pentile,E_C/O_decile,E_C/O_quartile,PIB_pentile,PIB_decile,PIB_quartile,Churned_Month,Churned_Year,Retail_Spend,Travel_Spend,Utilities_Spend,ProfSer_Spend,Entertainment_Spend,B2B_Spend,Retail_Cnt,Travel_Cnt,Utilities_Cnt,ProfSer_Cnt,Entertainment_Cnt,B2B_Cnt,Retail_Spend_ratio,Travel_Spend_ratio,Utilities_Spend_ratio,ProfSer_Spend_ratio,Entertainment_Spend_ratio,B2B_Spend_ratio,IS_Yes,PWP_flag,VFF_flag,AirForeign_flag,GiftCard_flag,PFC_flag,S&PWP_flag,Other_flag,Helloworld_flag,FX_flag,Webjet_flag,Hotel_flag,Airlocal_flag,Ent_flag,AirLocal_flag,AmexTravel_flag,RewardsCon_flag,QFF_flag,Donations_flag,Safekey_flag,all_MR_flag,retail_flag,Travel_flag,Utlities_flag,ProfSer_flag,B2B_flag,all_flag,acct_roc_decile,Churned_month,Churned_year,churne_month_year,acct_spend_decile_1,Adv_Publ,Air_Dom,Air_Intl,Air_Net,ATO,Car_Rental,Computer_Software,Consumables,Costco,Courier_Freight,Cruise,Dining,DJ_Prop_BB,DJ_GPC_BB,Education,Entertainment,Everything_Else,Financial_Services,Gas,Healthcare,Insurance,Lodging,Other_Air,Other_Gov,Other_Travel,Outbound,Personal_Services,QF_Air,Retail_DepStores,SBS_B2B,Supermarkets,Coles,Woolworths,Taxi_Limo,Telecomms,Tolls_Private,Transportation,Travel_Agents,Utilities,VA_Air,total,Adv_Publ_ratio,Air_Dom_ratio,Air_Intl_ratio,Air_Net_ratio,ATO_ratio,Car_Rental_ratio,Computer_Software_ratio,Consumables_ratio,Costco_ratio,Courier_Freight_ratio,Cruise_ratio,Dining_ratio,DJ_Prop_BB_ratio,DJ_GPC_BB_ratio,Education_ratio,Entertainment_ratio,Everything_Else_ratio,Financial_Services_ratio,Gas_ratio,Healthcare_ratio,Insurance_ratio,Lodging_ratio,Other_Air_ratio,Other_Gov_ratio,Other_Travel_ratio,Outbound_ratio,Personal_Services_ratio,QF_Air_ratio,Retail_DepStores_ratio,SBS_B2B_ratio,Supermarkets_ratio,Coles_ratio,Woolworths_ratio,Taxi_Limo_ratio,Telecomms_ratio,Tolls_Private_ratio,Transportation_ratio,Travel_Agents_ratio,Utilities_ratio,VA_Air_ratio,count,supp_status,cluster_label,all_zero_3year
0,0,LAV000000000000001,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,Unkn,U,NaN,1999-11-01,2014-09-01,NaN,Full Fee,NaN,NaN,GREEN,NaN,1,11,1999,1999-11-01,178,7. 10-15,1,-1,-1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33468.500000,0,0,1,NaN,NaN,-1,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,4,7,3,9.0,2014.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,9,2014,201409,-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,998.0,NaN
1,1,LA

In [69]:
xxx=DS_DF[mm]

In [68]:
mm=['LAV_ID1','Churned','tenure_bin','acct_spend_decile','supp_status','cluster_label','all_zero_3year','MR_Yes','PIB_Yes','all_MR_flag','PWP_flag', 'VFF_flag', 'AirForeign_flag','GiftCard_flag','PFC_flag','S&PWP_flag','Other_flag','Helloworld_flag',
       'FX_flag','Webjet_flag','Hotel_flag','Airlocal_flag','Ent_flag','AirLocal_flag','AmexTravel_flag','RewardsCon_flag','QFF_flag','Donations_flag','Safekey_flag']

In [70]:
xxx.shape

(1305584, 29)

In [72]:
df=df.merge(xxx,on='LAV_ID1',how='inner')

In [53]:
del df

In [39]:
del xxx

In [73]:
df.shape

(799324, 274)

In [74]:
df.head()

,LAV_ID1,cluster_label_x,MBR_STAT_CD,CM_ACCT_STAT_CD,product_description,RDM_pnt_Q1,RDM_pnt_Q2,RDM_pnt_Q3,RDM_pnt_Q4,RDM_cnt_Q1,RDM_cnt_Q2,RDM_cnt_Q3,RDM_cnt_Q4,All_RdPB_Q1,All_RdPB_Q2,All_RdPB_Q3,All_RdPB_Q4,PWP_pnt_Q1,PWP_pnt_Q2,PWP_pnt_Q3,PWP_pnt_Q4,VFF_pnt_Q1,VFF_pnt_Q2,VFF_pnt_Q3,VFF_pnt_Q4,Air - Foreign_pnt_Q1,Air - Foreign_pnt_Q2,Air - Foreign_pnt_Q3,Air - Foreign_pnt_Q4,Gift Card_pnt_Q1,Gift Card_pnt_Q2,Gift Card_pnt_Q3,Gift Card_pnt_Q4,Points for credit_pnt_Q1,Points for credit_pnt_Q2,Points for credit_pnt_Q3,Points for credit_pnt_Q4,S&PWP_pnt_Q1,S&PWP_pnt_Q2,S&PWP_pnt_Q3,S&PWP_pnt_Q4,Other_pnt_Q1,Other_pnt_Q2,Other_pnt_Q3,Other_pnt_Q4,Helloworld_pnt_Q1,Helloworld_pnt_Q2,Helloworld_pnt_Q3,Helloworld_pnt_Q4,FX_pnt_Q1,FX_pnt_Q2,FX_pnt_Q3,FX_pnt_Q4,Webjet_pnt_Q1,Webjet_pnt_Q2,Webjet_pnt_Q3,Webjet_pnt_Q4,Hotels_pnt_Q1,Hotels_pnt_Q2,Hotels_pnt_Q3,Hotels_pnt_Q4,Air - local_pnt_Q1,Air - local_pnt_Q2,Air - local_pnt_Q3,Air - local_pnt_Q4,Entertainment_pnt_Q1,Entertainment_pnt_Q2,Entertainment_pnt_Q3,Entertainment_pnt_Q4,Air - Local_pnt_Q1,Air - Local_pnt_Q2,Air - Local_pnt_Q3,Air - Local_pnt_Q4,Amex Travel Online_pnt_Q1,Amex Travel Online_pnt_Q2,Amex Travel Online_pnt_Q3,Amex Travel Online_pnt_Q4,Rewards Conceirge_pnt_Q1,Rewards Conceirge_pnt_Q2,Rewards Conceirge_pnt_Q3,Rewards Conceirge_pnt_Q4,QFF_pnt_Q1,QFF_pnt_Q2,QFF_pnt_Q3,QFF_pnt_Q4,Donations_pnt_Q1,Donations_pnt_Q2,Donations_pnt_Q3,Donations_pnt_Q4,Safekey_pnt_Q1,Safekey_pnt_Q2,Safekey_pnt_Q3,Safekey_pnt_Q4,PWP_cnt_Q1,PWP_cnt_Q2,PWP_cnt_Q3,PWP_cnt_Q4,VFF_cnt_Q1,VFF_cnt_Q2,VFF_cnt_Q3,VFF_cnt_Q4,Air - Foreign_cnt_Q1,Air - Foreign_cnt_Q2,Air - Foreign_cnt_Q3,Air - Foreign_cnt_Q4,Gift Card_cnt_Q1,Gift Card_cnt_Q2,Gift Card_cnt_Q3,Gift Card_cnt_Q4,Points for credit_cnt_Q1,Points for credit_cnt_Q2,Points for credit_cnt_Q3,Points for credit_cnt_Q4,S&PWP_cnt_Q1,S&PWP_cnt_Q2,S&PWP_cnt_Q3,S&PWP_cnt_Q4,Other_cnt_Q1,Other_cnt_Q2,Other_cnt_Q3,Other_cnt_Q4,Helloworld_cnt_Q1,Helloworld_cnt_Q2,Helloworld_cnt_Q3,Helloworld_cnt_Q4,FX_cnt_Q1,FX_cnt_Q2,FX_cnt_Q3,FX_cnt_Q4,Webjet_cnt_Q1,Webjet_cnt_Q2,Webjet_cnt_Q3,Webjet_cnt_Q4,Hotels_cnt_Q1,Hotels_cnt_Q2,Hotels_cnt_Q3,Hotels_cnt_Q4,Air - local_cnt_Q1,Air - local_cnt_Q2,Air - local_cnt_Q3,Air - local_cnt_Q4,Entertainment_cnt_Q1,Entertainment_cnt_Q2,Entertainment_cnt_Q3,Entertainment_cnt_Q4,Air - Local_cnt_Q1,Air - Local_cnt_Q2,Air - Local_cnt_Q3,Air - Local_cnt_Q4,Amex Travel Online_cnt_Q1,Amex Travel Online_cnt_Q2,Amex Travel Online_cnt_Q3,Amex Travel Online_cnt_Q4,Rewards Conceirge_cnt_Q1,Rewards Conceirge_cnt_Q2,Rewards Conceirge_cnt_Q3,Rewards Conceirge_cnt_Q4,QFF_cnt_Q1,QFF_cnt_Q2,QFF_cnt_Q3,QFF_cnt_Q4,Donations_cnt_Q1,Donations_cnt_Q2,Donations_cnt_Q3,Donations_cnt_Q4,Safekey_cnt_Q1,Safekey_cnt_Q2,Safekey_cnt_Q3,Safekey_cnt_Q4,PWP_RdPB_Q1,PWP_RdPB_Q2,PWP_RdPB_Q3,PWP_RdPB_Q4,VFF_RdPB_Q1,VFF_RdPB_Q2,VFF_RdPB_Q3,VFF_RdPB_Q4,Air - Foreign_RdPB_Q1,Air - Foreign_RdPB_Q2,Air - Foreign_RdPB_Q3,Air - Foreign_RdPB_Q4,Gift Card_RdPB_Q1,Gift Card_RdPB_Q2,Gift Card_RdPB_Q3,Gift Card_RdPB_Q4,Points for credit_RdPB_Q1,Points for credit_RdPB_Q2,Points for credit_RdPB_Q3,Points for credit_RdPB_Q4,S&PWP_RdPB_Q1,S&PWP_RdPB_Q2,S&PWP_RdPB_Q3,S&PWP_RdPB_Q4,Other_RdPB_Q1,Other_RdPB_Q2,Other_RdPB_Q3,Other_RdPB_Q4,Helloworld_RdPB_Q1,Helloworld_RdPB_Q2,Helloworld_RdPB_Q3,Helloworld_RdPB_Q4,FX_RdPB_Q1,FX_RdPB_Q2,FX_RdPB_Q3,FX_RdPB_Q4,Webjet_RdPB_Q1,Webjet_RdPB_Q2,Webjet_RdPB_Q3,Webjet_RdPB_Q4,Hotels_RdPB_Q1,Hotels_RdPB_Q2,Hotels_RdPB_Q3,Hotels_RdPB_Q4,Air - local_RdPB_Q1,Air - local_RdPB_Q2,Air - local_RdPB_Q3,Air - local_RdPB_Q4,Entertainment_RdPB_Q1,Entertainment_RdPB_Q2,Entertainment_RdPB_Q3,Entertainment_RdPB_Q4,Air - Local_RdPB_Q1,Air - Local_RdPB_Q2,Air - Local_RdPB_Q3,Air - Local_RdPB_Q4,Amex Travel Online_RdPB_Q1,Amex Travel Online_RdPB_Q2,Amex Travel Online_RdPB_Q3,Amex Travel Online_RdPB_Q4,Rewards Conceirge_RdPB_Q1,Rewards Conceirge_RdPB_Q2,Rewards Conceirge_RdPB_Q3,Rewards Conceirge_RdPB_Q4,QFF_RdPB_Q1,QFF_RdPB_Q2,QFF_RdPB_Q3,QFF_RdPB_Q4,Donations_R

In [76]:
df[df['QFF_flag']==1].groupby('acct_spend_decile').agg({'QFF_RdPB_Q1':'mean','QFF_RdPB_Q2':'mean','QFF_RdPB_Q3':'mean','QFF_RdPB_Q4':'mean'})

,QFF_RdPB_Q1,QFF_RdPB_Q2,QFF_RdPB_Q3,QFF_RdPB_Q4
acct_spend_decile,,,,
-1,0.428087,0.337558,0.262789,0.383051
1,0.376594,0.301134,0.284353,0.344511
2,0.460907,0.408991,0.429053,0.505407
3,0.545096,0.489464,0.516208,0.571135
4,0.551054,0.539783,0.558282,0.593320
5,0.570688,0.563883,0.582506,0.600070
6,0.586835,0.565538,0.585021,0.592724
7,0.586451,0.580226,0.578187,0.590572
8,0.587309,0.576601,0.575588,0.577627


In [79]:
df[['QFF_RdPB_Q1','QFF_RdPB_Q2','QFF_RdPB_Q3','QFF_RdPB_Q4']].mean()

QFF_RdPB_Q1    0.082742
QFF_RdPB_Q2    0.081024
QFF_RdPB_Q3    0.079042
QFF_RdPB_Q4    0.081549
dtype: float64

In [78]:
df[df['QFF_flag']==1][['QFF_RdPB_Q1','QFF_RdPB_Q2','QFF_RdPB_Q3','QFF_RdPB_Q4']].mean()

QFF_RdPB_Q1    0.536748
QFF_RdPB_Q2    0.525602
QFF_RdPB_Q3    0.512748
QFF_RdPB_Q4    0.529009
dtype: float64

In [80]:
df[df['GiftCard_flag']==1][['Gift Card_RdPB_Q1','Gift Card_RdPB_Q2','Gift Card_RdPB_Q3','Gift Card_RdPB_Q4']].mean()

Gift Card_RdPB_Q1    0.228514
Gift Card_RdPB_Q2    0.241678
Gift Card_RdPB_Q3    0.230320
Gift Card_RdPB_Q4    0.261423
dtype: float64

In [86]:
df[df['VFF_flag']==1][['VFF_RdPB_Q1','VFF_RdPB_Q2','VFF_RdPB_Q3','VFF_RdPB_Q4']].mean()

VFF_RdPB_Q1    0.193420
VFF_RdPB_Q2    0.427548
VFF_RdPB_Q3    0.234967
VFF_RdPB_Q4    0.376488
dtype: float64

In [87]:
df[df['S&PWP_flag']==1][['S&PWP_RdPB_Q1','S&PWP_RdPB_Q2','S&PWP_RdPB_Q3','S&PWP_RdPB_Q4']].mean()

S&PWP_RdPB_Q1    0.288685
S&PWP_RdPB_Q2    0.295606
S&PWP_RdPB_Q3    0.338653
S&PWP_RdPB_Q4    0.372675
dtype: float64

In [82]:
DS_DF.groupby('acct_spend_decile').agg({'acct_supp_spend_lcl':'sum'})

,acct_supp_spend_lcl
acct_spend_decile,
-1,6.070000e+02
1,2.804522e+04
2,1.572658e+05
3,3.803255e+05
4,8.087335e+05
5,1.625167e+06
6,3.082526e+06
7,6.226219e+06
8,1.301414e+07


In [83]:
df.head()

,LAV_ID1,cluster_label_x,MBR_STAT_CD,CM_ACCT_STAT_CD,product_description,RDM_pnt_Q1,RDM_pnt_Q2,RDM_pnt_Q3,RDM_pnt_Q4,RDM_cnt_Q1,RDM_cnt_Q2,RDM_cnt_Q3,RDM_cnt_Q4,All_RdPB_Q1,All_RdPB_Q2,All_RdPB_Q3,All_RdPB_Q4,PWP_pnt_Q1,PWP_pnt_Q2,PWP_pnt_Q3,PWP_pnt_Q4,VFF_pnt_Q1,VFF_pnt_Q2,VFF_pnt_Q3,VFF_pnt_Q4,Air - Foreign_pnt_Q1,Air - Foreign_pnt_Q2,Air - Foreign_pnt_Q3,Air - Foreign_pnt_Q4,Gift Card_pnt_Q1,Gift Card_pnt_Q2,Gift Card_pnt_Q3,Gift Card_pnt_Q4,Points for credit_pnt_Q1,Points for credit_pnt_Q2,Points for credit_pnt_Q3,Points for credit_pnt_Q4,S&PWP_pnt_Q1,S&PWP_pnt_Q2,S&PWP_pnt_Q3,S&PWP_pnt_Q4,Other_pnt_Q1,Other_pnt_Q2,Other_pnt_Q3,Other_pnt_Q4,Helloworld_pnt_Q1,Helloworld_pnt_Q2,Helloworld_pnt_Q3,Helloworld_pnt_Q4,FX_pnt_Q1,FX_pnt_Q2,FX_pnt_Q3,FX_pnt_Q4,Webjet_pnt_Q1,Webjet_pnt_Q2,Webjet_pnt_Q3,Webjet_pnt_Q4,Hotels_pnt_Q1,Hotels_pnt_Q2,Hotels_pnt_Q3,Hotels_pnt_Q4,Air - local_pnt_Q1,Air - local_pnt_Q2,Air - local_pnt_Q3,Air - local_pnt_Q4,Entertainment_pnt_Q1,Entertainment_pnt_Q2,Entertainment_pnt_Q3,Entertainment_pnt_Q4,Air - Local_pnt_Q1,Air - Local_pnt_Q2,Air - Local_pnt_Q3,Air - Local_pnt_Q4,Amex Travel Online_pnt_Q1,Amex Travel Online_pnt_Q2,Amex Travel Online_pnt_Q3,Amex Travel Online_pnt_Q4,Rewards Conceirge_pnt_Q1,Rewards Conceirge_pnt_Q2,Rewards Conceirge_pnt_Q3,Rewards Conceirge_pnt_Q4,QFF_pnt_Q1,QFF_pnt_Q2,QFF_pnt_Q3,QFF_pnt_Q4,Donations_pnt_Q1,Donations_pnt_Q2,Donations_pnt_Q3,Donations_pnt_Q4,Safekey_pnt_Q1,Safekey_pnt_Q2,Safekey_pnt_Q3,Safekey_pnt_Q4,PWP_cnt_Q1,PWP_cnt_Q2,PWP_cnt_Q3,PWP_cnt_Q4,VFF_cnt_Q1,VFF_cnt_Q2,VFF_cnt_Q3,VFF_cnt_Q4,Air - Foreign_cnt_Q1,Air - Foreign_cnt_Q2,Air - Foreign_cnt_Q3,Air - Foreign_cnt_Q4,Gift Card_cnt_Q1,Gift Card_cnt_Q2,Gift Card_cnt_Q3,Gift Card_cnt_Q4,Points for credit_cnt_Q1,Points for credit_cnt_Q2,Points for credit_cnt_Q3,Points for credit_cnt_Q4,S&PWP_cnt_Q1,S&PWP_cnt_Q2,S&PWP_cnt_Q3,S&PWP_cnt_Q4,Other_cnt_Q1,Other_cnt_Q2,Other_cnt_Q3,Other_cnt_Q4,Helloworld_cnt_Q1,Helloworld_cnt_Q2,Helloworld_cnt_Q3,Helloworld_cnt_Q4,FX_cnt_Q1,FX_cnt_Q2,FX_cnt_Q3,FX_cnt_Q4,Webjet_cnt_Q1,Webjet_cnt_Q2,Webjet_cnt_Q3,Webjet_cnt_Q4,Hotels_cnt_Q1,Hotels_cnt_Q2,Hotels_cnt_Q3,Hotels_cnt_Q4,Air - local_cnt_Q1,Air - local_cnt_Q2,Air - local_cnt_Q3,Air - local_cnt_Q4,Entertainment_cnt_Q1,Entertainment_cnt_Q2,Entertainment_cnt_Q3,Entertainment_cnt_Q4,Air - Local_cnt_Q1,Air - Local_cnt_Q2,Air - Local_cnt_Q3,Air - Local_cnt_Q4,Amex Travel Online_cnt_Q1,Amex Travel Online_cnt_Q2,Amex Travel Online_cnt_Q3,Amex Travel Online_cnt_Q4,Rewards Conceirge_cnt_Q1,Rewards Conceirge_cnt_Q2,Rewards Conceirge_cnt_Q3,Rewards Conceirge_cnt_Q4,QFF_cnt_Q1,QFF_cnt_Q2,QFF_cnt_Q3,QFF_cnt_Q4,Donations_cnt_Q1,Donations_cnt_Q2,Donations_cnt_Q3,Donations_cnt_Q4,Safekey_cnt_Q1,Safekey_cnt_Q2,Safekey_cnt_Q3,Safekey_cnt_Q4,PWP_RdPB_Q1,PWP_RdPB_Q2,PWP_RdPB_Q3,PWP_RdPB_Q4,VFF_RdPB_Q1,VFF_RdPB_Q2,VFF_RdPB_Q3,VFF_RdPB_Q4,Air - Foreign_RdPB_Q1,Air - Foreign_RdPB_Q2,Air - Foreign_RdPB_Q3,Air - Foreign_RdPB_Q4,Gift Card_RdPB_Q1,Gift Card_RdPB_Q2,Gift Card_RdPB_Q3,Gift Card_RdPB_Q4,Points for credit_RdPB_Q1,Points for credit_RdPB_Q2,Points for credit_RdPB_Q3,Points for credit_RdPB_Q4,S&PWP_RdPB_Q1,S&PWP_RdPB_Q2,S&PWP_RdPB_Q3,S&PWP_RdPB_Q4,Other_RdPB_Q1,Other_RdPB_Q2,Other_RdPB_Q3,Other_RdPB_Q4,Helloworld_RdPB_Q1,Helloworld_RdPB_Q2,Helloworld_RdPB_Q3,Helloworld_RdPB_Q4,FX_RdPB_Q1,FX_RdPB_Q2,FX_RdPB_Q3,FX_RdPB_Q4,Webjet_RdPB_Q1,Webjet_RdPB_Q2,Webjet_RdPB_Q3,Webjet_RdPB_Q4,Hotels_RdPB_Q1,Hotels_RdPB_Q2,Hotels_RdPB_Q3,Hotels_RdPB_Q4,Air - local_RdPB_Q1,Air - local_RdPB_Q2,Air - local_RdPB_Q3,Air - local_RdPB_Q4,Entertainment_RdPB_Q1,Entertainment_RdPB_Q2,Entertainment_RdPB_Q3,Entertainment_RdPB_Q4,Air - Local_RdPB_Q1,Air - Local_RdPB_Q2,Air - Local_RdPB_Q3,Air - Local_RdPB_Q4,Amex Travel Online_RdPB_Q1,Amex Travel Online_RdPB_Q2,Amex Travel Online_RdPB_Q3,Amex Travel Online_RdPB_Q4,Rewards Conceirge_RdPB_Q1,Rewards Conceirge_RdPB_Q2,Rewards Conceirge_RdPB_Q3,Rewards Conceirge_RdPB_Q4,QFF_RdPB_Q1,QFF_RdPB_Q2,QFF_RdPB_Q3,QFF_RdPB_Q4,Donations_R

In [84]:
pd.crosstab(df['MBR_STAT_CD'],df['MR_Yes'])

MR_Yes,0,1
MBR_STAT_CD,,
A,440095,267584
C,3737,2486
I,24829,1755


In [85]:
pd.crosstab(df['CM_ACCT_STAT_CD'],df['MR_Yes'])

MR_Yes,0,1
CM_ACCT_STAT_CD,,
A,464688,269458
C,3973,2367


In [89]:
DS_DF[DS_DF['Churned']==0].groupby('acct_spend_decile')['months_active'].agg('mean')

acct_spend_decile
-1     143.249436
 1     107.356887
 2      90.086453
 3      80.119277
 4      83.424288
 5      85.805169
 6      90.751678
 7      98.674941
 8     109.680669
 9     121.110541
 10    130.529985
Name: months_active, dtype: float64

In [90]:
ddd=DS_DF.pivot_table(index=['tenure_bin'], columns=['product_description'], values=['acct_spend_lcl'],aggfunc='sum').reset_index().fillna(value=0)

In [94]:
df.head()

,LAV_ID1,cluster_label_x,MBR_STAT_CD,CM_ACCT_STAT_CD,product_description,RDM_pnt_Q1,RDM_pnt_Q2,RDM_pnt_Q3,RDM_pnt_Q4,RDM_cnt_Q1,RDM_cnt_Q2,RDM_cnt_Q3,RDM_cnt_Q4,All_RdPB_Q1,All_RdPB_Q2,All_RdPB_Q3,All_RdPB_Q4,PWP_pnt_Q1,PWP_pnt_Q2,PWP_pnt_Q3,PWP_pnt_Q4,VFF_pnt_Q1,VFF_pnt_Q2,VFF_pnt_Q3,VFF_pnt_Q4,Air - Foreign_pnt_Q1,Air - Foreign_pnt_Q2,Air - Foreign_pnt_Q3,Air - Foreign_pnt_Q4,Gift Card_pnt_Q1,Gift Card_pnt_Q2,Gift Card_pnt_Q3,Gift Card_pnt_Q4,Points for credit_pnt_Q1,Points for credit_pnt_Q2,Points for credit_pnt_Q3,Points for credit_pnt_Q4,S&PWP_pnt_Q1,S&PWP_pnt_Q2,S&PWP_pnt_Q3,S&PWP_pnt_Q4,Other_pnt_Q1,Other_pnt_Q2,Other_pnt_Q3,Other_pnt_Q4,Helloworld_pnt_Q1,Helloworld_pnt_Q2,Helloworld_pnt_Q3,Helloworld_pnt_Q4,FX_pnt_Q1,FX_pnt_Q2,FX_pnt_Q3,FX_pnt_Q4,Webjet_pnt_Q1,Webjet_pnt_Q2,Webjet_pnt_Q3,Webjet_pnt_Q4,Hotels_pnt_Q1,Hotels_pnt_Q2,Hotels_pnt_Q3,Hotels_pnt_Q4,Air - local_pnt_Q1,Air - local_pnt_Q2,Air - local_pnt_Q3,Air - local_pnt_Q4,Entertainment_pnt_Q1,Entertainment_pnt_Q2,Entertainment_pnt_Q3,Entertainment_pnt_Q4,Air - Local_pnt_Q1,Air - Local_pnt_Q2,Air - Local_pnt_Q3,Air - Local_pnt_Q4,Amex Travel Online_pnt_Q1,Amex Travel Online_pnt_Q2,Amex Travel Online_pnt_Q3,Amex Travel Online_pnt_Q4,Rewards Conceirge_pnt_Q1,Rewards Conceirge_pnt_Q2,Rewards Conceirge_pnt_Q3,Rewards Conceirge_pnt_Q4,QFF_pnt_Q1,QFF_pnt_Q2,QFF_pnt_Q3,QFF_pnt_Q4,Donations_pnt_Q1,Donations_pnt_Q2,Donations_pnt_Q3,Donations_pnt_Q4,Safekey_pnt_Q1,Safekey_pnt_Q2,Safekey_pnt_Q3,Safekey_pnt_Q4,PWP_cnt_Q1,PWP_cnt_Q2,PWP_cnt_Q3,PWP_cnt_Q4,VFF_cnt_Q1,VFF_cnt_Q2,VFF_cnt_Q3,VFF_cnt_Q4,Air - Foreign_cnt_Q1,Air - Foreign_cnt_Q2,Air - Foreign_cnt_Q3,Air - Foreign_cnt_Q4,Gift Card_cnt_Q1,Gift Card_cnt_Q2,Gift Card_cnt_Q3,Gift Card_cnt_Q4,Points for credit_cnt_Q1,Points for credit_cnt_Q2,Points for credit_cnt_Q3,Points for credit_cnt_Q4,S&PWP_cnt_Q1,S&PWP_cnt_Q2,S&PWP_cnt_Q3,S&PWP_cnt_Q4,Other_cnt_Q1,Other_cnt_Q2,Other_cnt_Q3,Other_cnt_Q4,Helloworld_cnt_Q1,Helloworld_cnt_Q2,Helloworld_cnt_Q3,Helloworld_cnt_Q4,FX_cnt_Q1,FX_cnt_Q2,FX_cnt_Q3,FX_cnt_Q4,Webjet_cnt_Q1,Webjet_cnt_Q2,Webjet_cnt_Q3,Webjet_cnt_Q4,Hotels_cnt_Q1,Hotels_cnt_Q2,Hotels_cnt_Q3,Hotels_cnt_Q4,Air - local_cnt_Q1,Air - local_cnt_Q2,Air - local_cnt_Q3,Air - local_cnt_Q4,Entertainment_cnt_Q1,Entertainment_cnt_Q2,Entertainment_cnt_Q3,Entertainment_cnt_Q4,Air - Local_cnt_Q1,Air - Local_cnt_Q2,Air - Local_cnt_Q3,Air - Local_cnt_Q4,Amex Travel Online_cnt_Q1,Amex Travel Online_cnt_Q2,Amex Travel Online_cnt_Q3,Amex Travel Online_cnt_Q4,Rewards Conceirge_cnt_Q1,Rewards Conceirge_cnt_Q2,Rewards Conceirge_cnt_Q3,Rewards Conceirge_cnt_Q4,QFF_cnt_Q1,QFF_cnt_Q2,QFF_cnt_Q3,QFF_cnt_Q4,Donations_cnt_Q1,Donations_cnt_Q2,Donations_cnt_Q3,Donations_cnt_Q4,Safekey_cnt_Q1,Safekey_cnt_Q2,Safekey_cnt_Q3,Safekey_cnt_Q4,PWP_RdPB_Q1,PWP_RdPB_Q2,PWP_RdPB_Q3,PWP_RdPB_Q4,VFF_RdPB_Q1,VFF_RdPB_Q2,VFF_RdPB_Q3,VFF_RdPB_Q4,Air - Foreign_RdPB_Q1,Air - Foreign_RdPB_Q2,Air - Foreign_RdPB_Q3,Air - Foreign_RdPB_Q4,Gift Card_RdPB_Q1,Gift Card_RdPB_Q2,Gift Card_RdPB_Q3,Gift Card_RdPB_Q4,Points for credit_RdPB_Q1,Points for credit_RdPB_Q2,Points for credit_RdPB_Q3,Points for credit_RdPB_Q4,S&PWP_RdPB_Q1,S&PWP_RdPB_Q2,S&PWP_RdPB_Q3,S&PWP_RdPB_Q4,Other_RdPB_Q1,Other_RdPB_Q2,Other_RdPB_Q3,Other_RdPB_Q4,Helloworld_RdPB_Q1,Helloworld_RdPB_Q2,Helloworld_RdPB_Q3,Helloworld_RdPB_Q4,FX_RdPB_Q1,FX_RdPB_Q2,FX_RdPB_Q3,FX_RdPB_Q4,Webjet_RdPB_Q1,Webjet_RdPB_Q2,Webjet_RdPB_Q3,Webjet_RdPB_Q4,Hotels_RdPB_Q1,Hotels_RdPB_Q2,Hotels_RdPB_Q3,Hotels_RdPB_Q4,Air - local_RdPB_Q1,Air - local_RdPB_Q2,Air - local_RdPB_Q3,Air - local_RdPB_Q4,Entertainment_RdPB_Q1,Entertainment_RdPB_Q2,Entertainment_RdPB_Q3,Entertainment_RdPB_Q4,Air - Local_RdPB_Q1,Air - Local_RdPB_Q2,Air - Local_RdPB_Q3,Air - Local_RdPB_Q4,Amex Travel Online_RdPB_Q1,Amex Travel Online_RdPB_Q2,Amex Travel Online_RdPB_Q3,Amex Travel Online_RdPB_Q4,Rewards Conceirge_RdPB_Q1,Rewards Conceirge_RdPB_Q2,Rewards Conceirge_RdPB_Q3,Rewards Conceirge_RdPB_Q4,QFF_RdPB_Q1,QFF_RdPB_Q2,QFF_RdPB_Q3,QFF_RdPB_Q4,Donations_R

In [95]:
df.shape

(799324, 274)

In [101]:
df.columns

Index(['LAV_ID1', 'cluster_label_x', 'MBR_STAT_CD', 'CM_ACCT_STAT_CD',
       'product_description', 'RDM_pnt_Q1', 'RDM_pnt_Q2', 'RDM_pnt_Q3',
       'RDM_pnt_Q4', 'RDM_cnt_Q1',
       ...
       'Webjet_flag', 'Hotel_flag', 'Airlocal_flag', 'Ent_flag',
       'AirLocal_flag', 'AmexTravel_flag', 'RewardsCon_flag', 'QFF_flag',
       'Donations_flag', 'Safekey_flag'],
      dtype='object', length=274)

In [97]:
DS_DF.shape

(1305584, 232)

In [100]:
df.to_csv('/Users/sri.t/Desktop/MR_prepared_by_Level2.csv',header=True,sep=',',encoding='utf-8')

In [98]:
DS_DF.to_csv('/Users/sri.t/Desktop/amex_churners_estimates_4.csv',header=True,sep=',',encoding='utf-8')

In [108]:
del df